<a href="https://colab.research.google.com/github/WagnerZoega/bet365-test/blob/main/pool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cupy-cuda11x coincurve eth-utils base58 pycryptodome
!apt install cupy-cuda11x coincurve eth-utils base58 pycryptodome

In [ ]:
"""
Script para instalar dependências CUDA corretas para o Google Colab
Resolve problemas de compatibilidade de biblioteca como o libnvrtc.so não encontrado
"""
import os
import sys
import subprocess
import glob
import re

def check_cuda_version():
    """Verifica a versão do CUDA instalada no sistema"""
    try:
        # Executa o comando nvcc --version
        result = subprocess.run(['nvcc', '--version'], stdout=subprocess.PIPE, text=True)
        output = result.stdout

        # Extrai a versão do CUDA usando regex
        match = re.search(r'release (\d+\.\d+)', output)
        if match:
            version = match.group(1)
            print(f"✅ Versão do CUDA detectada: {version}")
            return version
        else:
            # Tenta com nvidia-smi
            result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, text=True)
            output = result.stdout
            match = re.search(r'CUDA Version: (\d+\.\d+)', output)
            if match:
                version = match.group(1)
                print(f"✅ Versão do CUDA detectada (via nvidia-smi): {version}")
                return version

            print("⚠️ Não foi possível determinar a versão CUDA, assumindo 12.0")
            return "12.0"  # Padrão para Colab recentes
    except Exception as e:
        print(f"⚠️ Erro ao verificar versão CUDA: {e}")
        print("⚠️ Assumindo CUDA 12.0")
        return "12.0"  # Padrão para Colab recentes

def install_correct_cupy():
    """Instala a versão correta do CuPy baseado na versão CUDA"""
    cuda_version = check_cuda_version()
    major_version = int(cuda_version.split('.')[0])

    print(f"\n🔄 Instalando CuPy compatível com CUDA {cuda_version}...")

    # Primeiro remover qualquer instalação existente para evitar conflitos
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'cupy', 'cupy-cuda11x', 'cupy-cuda12x'])

    # Limpar o cache pip para garantir instalação limpa
    subprocess.run([sys.executable, '-m', 'pip', 'cache', 'purge'])

    # Instalar a versão correta do CuPy
    if major_version >= 12:
        print("🔄 Instalando cupy-cuda12x...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-cache-dir', 'cupy-cuda12x'])
    elif major_version == 11:
        print("🔄 Instalando cupy-cuda11x...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-cache-dir', 'cupy-cuda11x'])
    else:
        print(f"⚠️ Versão CUDA {cuda_version} pode não ser compatível com CuPy atual")
        print("🔄 Tentando instalar cupy-cuda11x como fallback...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-cache-dir', 'cupy-cuda11x'])

    # Verifica se CuPy foi instalado corretamente
    print("🔍 Verificando instalação do CuPy...")
    try:
        subprocess.run([sys.executable, '-c', 'import cupy; print(\"CuPy importado com sucesso\")'], check=True)
        print("✅ CuPy instalado corretamente")
    except subprocess.CalledProcessError:
        print("⚠️ CuPy não foi instalado corretamente")
        print("⚠️ Tentando novamente com CuPy genérico...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-cache-dir', 'cupy'])

def setup_symbolic_links():
    """Configura links simbólicos para bibliotecas CUDA se necessário"""
    print("\n🔄 Configurando links simbólicos para bibliotecas CUDA...")

    try:
        # Encontrar arquivos libnvrtc*.so no sistema
        nvrtc_libs = []
        search_dirs = [
            '/usr/local/cuda/lib64/',
            '/usr/lib/x86_64-linux-gnu/',
            '/usr/lib/',
            '/usr/local/lib/'
        ]

        for directory in search_dirs:
            if os.path.exists(directory):
                nvrtc_libs.extend(glob.glob(f"{directory}libnvrtc*.so*"))

        if nvrtc_libs:
            print(f"📋 Bibliotecas NVRTC encontradas: {len(nvrtc_libs)}")
            for lib in nvrtc_libs[:5]:  # Mostra até 5 para não sobrecarregar a saída
                print(f"   - {lib}")

            # Cria link simbólico para libnvrtc.so.11.2 se necessário
            if not any('libnvrtc.so.11.2' in lib for lib in nvrtc_libs):
                # Encontra a biblioteca mais recente para usar como alvo
                target_lib = None
                for lib in nvrtc_libs:
                    if os.path.islink(lib) and not os.path.exists(lib):
                        continue  # Pula links quebrados
                    target_lib = lib
                    break

                if target_lib:
                    # Criar diretório de links
                    os.makedirs('/tmp/cuda_links/', exist_ok=True)
                    link_path = '/tmp/cuda_links/libnvrtc.so.11.2'

                    # Remover link existente se necessário
                    if os.path.exists(link_path):
                        os.remove(link_path)

                    # Criar link simbólico
                    os.symlink(target_lib, link_path)
                    print(f"✅ Link simbólico criado: {link_path} -> {target_lib}")

                    # Adicionar ao LD_LIBRARY_PATH
                    os.environ['LD_LIBRARY_PATH'] = f"/tmp/cuda_links:{os.environ.get('LD_LIBRARY_PATH', '')}"
                    print(f"✅ LD_LIBRARY_PATH atualizado: {os.environ['LD_LIBRARY_PATH']}")
                else:
                    print("⚠️ Não foi possível encontrar uma biblioteca NVRTC válida para criar link")
            else:
                print("✅ libnvrtc.so.11.2 já existe, não é necessário criar link")
        else:
            print("⚠️ Nenhuma biblioteca NVRTC encontrada no sistema")

    except Exception as e:
        print(f"⚠️ Erro ao configurar links simbólicos: {e}")

def install_other_dependencies():
    """Instala outras dependências necessárias"""
    print("\n🔄 Instalando outras dependências...")

    dependencies = [
        "coincurve",
        "eth-utils",
        "base58",
        "torch"
    ]

    for dep in dependencies:
        print(f"🔄 Instalando {dep}...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', dep])

def check_installation():
    """Verifica se a instalação foi bem-sucedida"""
    print("\n🔍 Verificando instalação...")

    try:
        print("🔄 Importando CuPy...")
        import cupy as cp

        # Obtém a versão de uma maneira mais segura
        try:
            # Tenta diferentes maneiras de obter a versão
            version = None
            if hasattr(cp, '__version__'):
                version = cp.__version__
            elif hasattr(cp, 'version'):
                version = cp.version
            elif hasattr(cp, 'core') and hasattr(cp.core, 'CUPY_VERSION'):
                version = cp.core.CUPY_VERSION

            if version:
                print(f"✅ CuPy versão {version} importado com sucesso")
            else:
                print("✅ CuPy importado com sucesso (versão não disponível)")
        except:
            print("✅ CuPy importado com sucesso (não foi possível determinar a versão)")

        if cp.cuda.is_available():
            print("✅ CUDA disponível via CuPy!")
            try:
                device_props = cp.cuda.runtime.getDeviceProperties(0)
                print(f"   Dispositivo: {device_props['name'].decode()}")
            except:
                print(f"   (Não foi possível obter o nome do dispositivo)")

            try:
                mem = cp.cuda.runtime.memGetInfo()
                print(f"   Memória livre: {mem[0]/1024**3:.2f} GB / {mem[1]/1024**3:.2f} GB")
            except:
                print("   (Não foi possível obter informações de memória)")

            # Teste rápido
            print("\n🔄 Executando teste rápido de GPU...")
            try:
                x = cp.arange(10)
                y = cp.arange(10)
                z = x + y
                print(f"✅ Teste concluído: {z.get()}")
                return True
            except Exception as e:
                print(f"⚠️ Erro ao executar teste simples: {e}")
                print("   Isto pode indicar problemas com o runtime CUDA")
                return False
        else:
            print("❌ CUDA não está disponível via CuPy")
            try:
                print("\nVerificando problema:")
                print(f"   CUDA disponibilidade reportada: {cp.cuda.is_available()}")
                print(f"   Número de dispositivos: {cp.cuda.runtime.getDeviceCount()}")
            except Exception as e:
                print(f"   Erro ao verificar dispositivos: {e}")
            return False

    except Exception as e:
        print(f"❌ Erro ao verificar instalação: {e}")
        print("\nInformações de depuração:")
        try:
            import sys
            print(f"Python: {sys.version}")
            print(f"Localização do pacote CuPy:")

            # Tenta localizar o pacote CuPy mesmo com erro
            try:
                import importlib.util
                cupy_spec = importlib.util.find_spec("cupy")
                if cupy_spec:
                    print(f"   Encontrado em: {cupy_spec.origin}")
                else:
                    print("   Pacote não encontrado no sistema")
            except:
                print("   Não foi possível localizar o pacote")

            # Verifica ambiente CUDA
            print("\nAmbiente CUDA:")
            import os
            for var in ['CUDA_HOME', 'CUDA_PATH', 'LD_LIBRARY_PATH']:
                print(f"   {var}: {os.environ.get(var, 'não definido')}")

            # Tenta mostrar bibliotecas disponíveis
            try:
                import subprocess
                result = subprocess.run('ldconfig -p | grep cuda', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                if result.returncode == 0 and result.stdout:
                    print("\nBibliotecas CUDA no sistema:")
                    for line in result.stdout.splitlines()[:10]:  # Mostra até 10 linhas
                        print(f"   {line}")
                    if len(result.stdout.splitlines()) > 10:
                        print(f"   ... mais {len(result.stdout.splitlines()) - 10} bibliotecas")
            except:
                pass
        except:
            print("   Não foi possível coletar informações de depuração adicionais")

        import traceback
        traceback.print_exc()
        return False

def main():
    print("=" * 60)
    print("🔧 CONFIGURAÇÃO CUDA PARA BITCOINFLIX MINER 🔧")
    print("=" * 60)

    # Verificar se estamos no Google Colab
    in_colab = 'google.colab' in sys.modules
    if in_colab:
        print("✅ Ambiente Google Colab detectado")

        # Verificar se runtime Colab está com GPU
        try:
            import torch
            if not torch.cuda.is_available():
                print("\n❌ IMPORTANTE: GPU NÃO DETECTADA NO COLAB!")
                print("   Certifique-se de que selecionou GPU em: Runtime > Change runtime type")
                proceed = input("Continuar mesmo sem GPU? (s/n): ")
                if proceed.lower() != 's':
                    print("Abortando instalação.")
                    return
            else:
                print(f"✅ GPU detectada: {torch.cuda.get_device_name(0)}")
        except:
            print("\n⚠️ Não foi possível verificar GPU via PyTorch")
    else:
        print("⚠️ Este script é otimizado para Google Colab")

    # Instalar versão correta do CuPy
    install_correct_cupy()

    # Configurar links simbólicos
    setup_symbolic_links()

    # Instalar outras dependências
    install_other_dependencies()

    # Verificar instalação
    success = check_installation()

    print("\n" + "=" * 60)
    if success:
        print("✅ CONFIGURAÇÃO CONCLUÍDA COM SUCESSO!")
        print("🚀 Agora você pode executar colab_cuda_alt.py")
    else:
        print("⚠️ CONFIGURAÇÃO COM PROBLEMAS")
        print("❓ Tente uma das seguintes opções:")
        print("   1. Reinicie o runtime (Runtime > Restart runtime) e execute este script novamente")
        print("   2. Verifique se você selecionou GPU em Runtime > Change runtime type")
        print("   3. Tente executar colab_cuda_alt.py mesmo assim - a verificação pode falhar mas o script pode funcionar")
    print("=" * 60)

if __name__ == "__main__":
    main()


In [ ]:
"""
Script para resolver problemas de compatibilidade do CuPy com CUDA
Resolve os seguintes problemas:
1. Múltiplos pacotes CuPy instalados (cupy-cuda11x e cupy-cuda12x)
2. Link simbólico faltando para libnvrtc.so.11.2
"""
import os
import sys
import subprocess
import glob
import re
import time

def check_environment():
    """Verifica o ambiente e mostra informações relevantes"""
    print("=" * 60)
    print("🔍 VERIFICANDO AMBIENTE CUDA/CuPy")
    print("=" * 60)

    # Verificar se estamos no Google Colab
    is_colab = 'google.colab' in sys.modules
    if is_colab:
        print("✅ Ambiente Google Colab detectado")
    else:
        print("ℹ️ Executando fora do Google Colab")

    # Verificar versão do CUDA via nvidia-smi
    try:
        print("\n📋 Verificando versão CUDA...")
        result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
        cuda_match = re.search(r'CUDA Version: (\d+\.\d+)', result.stdout)
        if cuda_match:
            cuda_version = cuda_match.group(1)
            print(f"✅ CUDA versão {cuda_version} detectada")
            major_version = cuda_version.split('.')[0]
            print(f"   Versão principal: {major_version}")
        else:
            print("⚠️ Não foi possível determinar a versão do CUDA via nvidia-smi")
            cuda_version = None
    except Exception as e:
        print(f"⚠️ Erro ao executar nvidia-smi: {e}")
        cuda_version = None

    # Verificar pacotes CuPy instalados
    try:
        print("\n📋 Verificando instalações do CuPy...")
        result = subprocess.run([sys.executable, "-m", "pip", "list"], capture_output=True, text=True)
        cupy_packages = re.findall(r'(cupy[^\s]+)\s+([^\s]+)', result.stdout)

        if cupy_packages:
            print(f"📦 Pacotes CuPy instalados:")
            for pkg, version in cupy_packages:
                print(f"   - {pkg} {version}")
        else:
            print("⚠️ CuPy não está instalado")
    except Exception as e:
        print(f"⚠️ Erro ao verificar pacotes instalados: {e}")

    # Verificar bibliotecas CUDA disponíveis
    try:
        print("\n📋 Verificando bibliotecas CUDA...")
        library_dirs = [
            "/usr/local/cuda/lib64",
            "/usr/lib/x86_64-linux-gnu",
            "/usr/lib",
            "/lib"
        ]

        nvrtc_libs = []
        cuda_libs = []

        for directory in library_dirs:
            if os.path.exists(directory):
                nvrtc_candidates = glob.glob(f"{directory}/libnvrtc*")
                nvrtc_libs.extend(nvrtc_candidates)

                # Busca por outras libs CUDA importantes
                cuda_candidates = glob.glob(f"{directory}/libcuda*") + glob.glob(f"{directory}/libcudart*")
                cuda_libs.extend(cuda_candidates)

        if nvrtc_libs:
            print(f"📚 Bibliotecas NVRTC encontradas ({len(nvrtc_libs)}):")
            for lib in nvrtc_libs[:5]:  # Mostrar apenas as primeiras 5
                print(f"   - {os.path.basename(lib)}")
            if len(nvrtc_libs) > 5:
                print(f"   - ... e mais {len(nvrtc_libs) - 5} bibliotecas")
        else:
            print("⚠️ Nenhuma biblioteca libnvrtc*.so encontrada")

        if cuda_libs:
            print(f"📚 Outras bibliotecas CUDA importantes ({len(cuda_libs)}):")
            for lib in cuda_libs[:5]:  # Mostrar apenas as primeiras 5
                print(f"   - {os.path.basename(lib)}")
            if len(cuda_libs) > 5:
                print(f"   - ... e mais {len(cuda_libs) - 5} bibliotecas")

        # Verificar a biblioteca específica que está causando o problema
        target_lib = "libnvrtc.so.11.2"
        target_found = False
        for path in nvrtc_libs:
            if target_lib in path:
                print(f"✅ {target_lib} encontrada: {path}")
                target_found = True
                break

        if not target_found:
            print(f"❌ {target_lib} não encontrada! Precisamos configurar um link simbólico.")
    except Exception as e:
        print(f"⚠️ Erro ao verificar bibliotecas: {e}")

    return cuda_version

def clean_cupy_installation():
    """Remove todas as instalações existentes do CuPy"""
    print("\n" + "=" * 60)
    print("🧹 LIMPANDO INSTALAÇÕES CuPy EXISTENTES")
    print("=" * 60)

    # Lista de pacotes CuPy para remover
    cupy_packages = ["cupy", "cupy-cuda11x", "cupy-cuda12x", "cupy-cuda110", "cupy-cuda111", "cupy-cuda112", "cupy-cuda120"]

    for pkg in cupy_packages:
        print(f"🗑️ Removendo {pkg}...")
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], stdout=subprocess.DEVNULL)

    # Limpar o cache pip para evitar problemas
    print("🗑️ Limpando cache pip...")
    subprocess.run([sys.executable, "-m", "pip", "cache", "purge"], stdout=subprocess.DEVNULL)

    print("✅ Todas as instalações do CuPy foram removidas")

def install_correct_cupy(cuda_version):
    """Instala a versão correta do CuPy com base na versão do CUDA"""
    print("\n" + "=" * 60)
    print("📦 INSTALANDO CuPy COMPATÍVEL")
    print("=" * 60)

    if cuda_version:
        major_version = int(float(cuda_version))
    else:
        # Tenta uma detecção alternativa
        print("⚠️ Não foi possível determinar a versão CUDA, tentando detectar novamente...")
        try:
            # Verificar via nvcc ou buscar outros indicadores
            try:
                result = subprocess.run(["nvcc", "--version"], capture_output=True, text=True)
                m = re.search(r'release (\d+\.\d+)', result.stdout)
                if m:
                    major_version = int(float(m.group(1)))
                    print(f"✅ CUDA {major_version} detectado via nvcc")
                else:
                    # Verificar presença de bibliotecas específicas
                    if os.path.exists("/usr/local/cuda-12"):
                        major_version = 12
                        print("✅ CUDA 12 detectado via diretórios")
                    elif os.path.exists("/usr/local/cuda-11"):
                        major_version = 11
                        print("✅ CUDA 11 detectado via diretórios")
                    else:
                        # Assumir versão mais recente
                        major_version = 12
                        print("⚠️ Versão CUDA não detectada, assumindo CUDA 12")
            except:
                # Assumir versão mais recente como fallback
                major_version = 12
                print("⚠️ Versão CUDA não detectada, assumindo CUDA 12")
        except:
            major_version = 12
            print("⚠️ Versão CUDA não detectada, assumindo CUDA 12")

    # Instalar a versão correta do CuPy
    if major_version >= 12:
        cupy_package = "cupy-cuda12x"
    elif major_version == 11:
        cupy_package = "cupy-cuda11x"
    else:
        print(f"⚠️ CUDA {major_version} pode não ser compatível com o CuPy atual!")
        print("   Tentando instalar a versão para CUDA 11.x como fallback")
        cupy_package = "cupy-cuda11x"

    print(f"📦 Instalando {cupy_package}...")
    start_time = time.time()
    try:
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "--no-cache-dir", "-v", cupy_package],
            capture_output=True,
            text=True
        )

        # Verificar erros específicos conhecidos
        if "ERROR: No matching distribution found for cupy" in result.stderr:
            print(f"❌ Erro: Pacote {cupy_package} não encontrado!")
            print("   Tentando com instalação genérica do cupy...")
            subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", "cupy"])
        elif result.returncode != 0:
            print(f"⚠️ Aviso: Possíveis problemas na instalação: {result.stderr}")
        else:
            print(f"✅ {cupy_package} instalado com sucesso!")
    except Exception as e:
        print(f"❌ Erro na instalação: {e}")
        print("   Tentando alternativa...")
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", "cupy"])
        except:
            print("❌ Todas as tentativas de instalação falharam")

    elapsed = time.time() - start_time
    print(f"⏱️ Instalação concluída em {elapsed:.1f} segundos")

def setup_symbolic_links():
    """Configura links simbólicos necessários para as bibliotecas CUDA"""
    print("\n" + "=" * 60)
    print("🔗 CONFIGURANDO LINKS SIMBÓLICOS")
    print("=" * 60)

    # Criar diretório para links simbólicos
    link_dir = "/tmp/cuda_links"
    os.makedirs(link_dir, exist_ok=True)
    print(f"📁 Diretório para links simbólicos: {link_dir}")

    # Adicionar ao LD_LIBRARY_PATH
    os.environ["LD_LIBRARY_PATH"] = f"{link_dir}:{os.environ.get('LD_LIBRARY_PATH', '')}"
    print(f"✅ LD_LIBRARY_PATH atualizado: {os.environ['LD_LIBRARY_PATH']}")

    # Encontrar bibliotecas NVRTC disponíveis
    library_dirs = [
        "/usr/local/cuda/lib64",
        "/usr/lib/x86_64-linux-gnu",
        "/usr/lib",
        "/lib"
    ]

    nvrtc_libs = []
    for directory in library_dirs:
        if os.path.exists(directory):
            nvrtc_candidates = glob.glob(f"{directory}/libnvrtc*")
            nvrtc_libs.extend(nvrtc_candidates)

    # Definir os links necessários e suas origens
    needed_links = {
        "libnvrtc.so.11.2": None  # Vai ser preenchido com a biblioteca encontrada
    }

    # Encontrar o melhor candidato para cada link
    for lib in nvrtc_libs:
        lib_name = os.path.basename(lib)

        # Para libnvrtc.so.11.2, queremos a versão mais próxima
        if "libnvrtc.so" in lib_name:
            # Já encontrou o arquivo exato?
            if lib_name == "libnvrtc.so.11.2":
                needed_links["libnvrtc.so.11.2"] = lib
                break

            # Se não temos um candidato ou este é um candidato melhor
            if needed_links["libnvrtc.so.11.2"] is None:
                needed_links["libnvrtc.so.11.2"] = lib

    # Criar links simbólicos
    for link_name, source_lib in needed_links.items():
        if source_lib:
            link_path = f"{link_dir}/{link_name}"

            # Remover link antigo se existir
            if os.path.exists(link_path):
                os.remove(link_path)

            os.symlink(source_lib, link_path)
            print(f"🔗 Link criado: {link_path} -> {source_lib}")
        else:
            print(f"❌ Não foi possível encontrar biblioteca para {link_name}!")
            print("   Isso pode causar falha ao usar CuPy!")

def verify_installation():
    """Verifica se a instalação do CuPy está funcionando corretamente"""
    print("\n" + "=" * 60)
    print("🔍 VERIFICANDO INSTALAÇÃO")
    print("=" * 60)

    try:
        print("🔄 Tentando importar CuPy...")
        import cupy as cp

        print("✅ CuPy importado com sucesso!")

        # Mostrar versão
        if hasattr(cp, "__version__"):
            print(f"📋 Versão: {cp.__version__}")
        else:
            print("⚠️ Não foi possível determinar a versão do CuPy")

        # Verificar CUDA disponível
        if cp.cuda.is_available():
            print("✅ CUDA disponível!")

            # Mostrar informações do dispositivo
            try:
                device = cp.cuda.Device(0)
                props = cp.cuda.runtime.getDeviceProperties(0)
                print(f"📊 Dispositivo: {props['name'].decode()}")
                print(f"   Memória total: {props['totalGlobalMem'] / (1024**3):.2f} GB")
                print(f"   Compute capability: {props['major']}.{props['minor']}")
            except Exception as e:
                print(f"⚠️ Erro ao obter informações do dispositivo: {e}")

            # Testar operações básicas
            print("\n🧪 Executando teste simples...")
            try:
                x = cp.array([1, 2, 3])
                y = cp.array([4, 5, 6])
                z = x + y
                print(f"   Resultado: {z}")
                print("✅ Teste bem-sucedido!")
                return True
            except Exception as e:
                print(f"❌ Teste falhou: {e}")
                return False
        else:
            print("❌ CUDA não está disponível!")
            return False
    except ImportError:
        print("❌ Não foi possível importar CuPy!")
        return False
    except Exception as e:
        print(f"❌ Erro ao verificar instalação: {e}")
        return False

def main():
    print("=" * 60)
    print("🛠️ CORREÇÃO DE PROBLEMAS CUDA/CuPy")
    print("=" * 60)

    # 1. Verificar ambiente
    cuda_version = check_environment()

    # 2. Limpar instalações do CuPy
    clean_cupy_installation()

    # 3. Instalar versão correta do CuPy
    install_correct_cupy(cuda_version)

    # 4. Configurar links simbólicos
    setup_symbolic_links()

    # 5. Verificar instalação
    success = verify_installation()

    print("\n" + "=" * 60)
    if success:
        print("✅ CONFIGURAÇÃO CONCLUÍDA COM SUCESSO!")
        print("\n🚀 Agora você pode executar batch_size_tester.py novamente.")
        print("   Recomendamos reiniciar o runtime antes de executar.")
    else:
        print("⚠️ CONFIGURAÇÃO CONCLUÍDA COM POSSÍVEIS PROBLEMAS")
        print("\n🔄 Por favor, siga estes passos:")
        print("   1. Reinicie o runtime (Runtime > Restart runtime)")
        print("   2. Execute novamente este script para verificar")
        print("   3. Tente executar batch_size_tester.py")
    print("=" * 60)

if __name__ == "__main__":
    main()


In [ ]:
"""
Este script testa vários tamanhos de batch para encontrar a configuração ideal
para o seu hardware específico de GPU.
"""
import time
import numpy as np
import sys
import subprocess
import os

# Verificar se estamos no ambiente do Colab ou Jupyter
IS_COLAB = 'google.colab' in sys.modules
IS_NOTEBOOK = 'ipykernel' in sys.modules

# Função para verificar e consertar o ambiente CUDA/CuPy
def check_and_fix_cupy():
    """Verifica se o ambiente CuPy está configurado corretamente e tenta corrigir se necessário."""
    # Verificar se temos múltiplos pacotes CuPy instalados
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "list"], capture_output=True, text=True)
        cupy_packages = [line for line in result.stdout.split('\n') if 'cupy' in line.lower()]

        if len(cupy_packages) > 1:
            print("⚠️ Detectados múltiplos pacotes CuPy instalados. Isso pode causar conflitos.")
            print("   Deseja executar o script de correção CUDA/CuPy? (Recomendado)")
            choice = input("   Executar cuda_setup_fix.py? (s/n): ").strip().lower()

            if choice == 's':
                # Verifica se o script de correção existe
                if os.path.exists("cuda_setup_fix.py"):
                    print("🔄 Executando script de correção...")
                    subprocess.run([sys.executable, "cuda_setup_fix.py"])
                    print("\n⚠️ Por favor, reinicie o runtime e execute batch_size_tester.py novamente.")
                    sys.exit(0)
                else:
                    print("❌ Script cuda_setup_fix.py não encontrado.")
                    print("   Tente reinstalar o CuPy manualmente:")
                    print("   !pip uninstall -y cupy cupy-cuda11x cupy-cuda12x")
                    print("   !pip install cupy-cuda12x  # Ou a versão apropriada")
                    return False

        # Verificar se libnvrtc.so.11.2 está disponível
        # Podemos tentar configurar os links simbólicos também
        try:
            # Testar se o CuPy consegue fazer operações básicas
            import cupy as cp
            test_array = cp.array([1, 2, 3])
            test_result = test_array + test_array
            return True
        except ImportError:
            print("❌ CuPy não está instalado.")
            print("   Tente instalar com: !pip install cupy-cuda12x")
            return False
        except Exception as e:
            if "libnvrtc.so.11.2" in str(e):
                print("⚠️ Erro de biblioteca libnvrtc.so.11.2 detectado.")
                print("   Executando configuração de links simbólicos...")

                try:
                    # Criar diretório para links e configurar
                    os.makedirs('/tmp/cuda_links', exist_ok=True)

                    # Procurar por libnvrtc em locais comuns
                    nvrtc_paths = []
                    for path in ["/usr/local/cuda/lib64", "/usr/lib/x86_64-linux-gnu"]:
                        if os.path.exists(path):
                            nvrtc_paths.extend(subprocess.run(f"find {path} -name 'libnvrtc.so*'",
                                                           shell=True,
                                                           capture_output=True,
                                                           text=True).stdout.splitlines())

                    if nvrtc_paths:
                        # Criar link usando a primeira biblioteca encontrada
                        target_path = nvrtc_paths[0]
                        link_path = "/tmp/cuda_links/libnvrtc.so.11.2"

                        # Remover link antigo, se existir
                        if os.path.exists(link_path):
                            os.remove(link_path)

                        # Criar novo link
                        os.symlink(target_path, link_path)

                        # Atualizar LD_LIBRARY_PATH
                        os.environ["LD_LIBRARY_PATH"] = f"/tmp/cuda_links:{os.environ.get('LD_LIBRARY_PATH', '')}"
                        print(f"✅ Link simbólico criado: {link_path} -> {target_path}")
                        print(f"✅ LD_LIBRARY_PATH atualizado")

                        print("\n⚠️ Por favor, reinicie o runtime e execute batch_size_tester.py novamente.")
                        sys.exit(0)
                    else:
                        print("❌ Não foi possível encontrar bibliotecas libnvrtc.so")
                        print("   Execute o script cuda_setup_fix.py ou reinstale o CuPy manualmente.")
                        return False
                except Exception as link_error:
                    print(f"❌ Erro ao configurar links simbólicos: {link_error}")
                    return False
            else:
                print(f"❌ Erro ao inicializar CuPy: {e}")
                return False
    except Exception as e:
        print(f"❌ Erro ao verificar ambiente CuPy: {e}")
        return False

# Verificar ambiente CuPy e corrigir se necessário
check_and_fix_cupy()

# Tentar importar bibliotecas necessárias
try:
    import cupy as cp
    HAS_CUPY = True
    print("✅ CuPy disponível")
except ImportError:
    HAS_CUPY = False
    print("❌ CuPy não disponível")
except Exception as e:
    print(f"❌ Erro ao inicializar CuPy: {e}")
    print("   Execute o script cuda_setup_fix.py para resolver problemas de configuração.")
    print("   Depois reinicie o runtime e execute este script novamente.")
    sys.exit(1)

try:
    from coincurve import PublicKey
    from eth_utils import keccak
    import base58
except ImportError:
    print("⚠️ Bibliotecas necessárias não encontradas.")
    print("   Instalando bibliotecas essenciais...")
    try:
        import subprocess
        subprocess.run([sys.executable, "-m", "pip", "install", "coincurve", "eth-utils", "base58"],
                       check=True)
        from coincurve import PublicKey
        from eth_utils import keccak
        import base58
        print("✅ Bibliotecas instaladas com sucesso")
    except Exception as e:
        print(f"❌ Erro ao instalar bibliotecas: {e}")
        sys.exit(1)

# Funções para benchmark
def custom_keccak(data):
    """Calcula Keccak-256."""
    try:
        if isinstance(data, str):
            data = bytes.fromhex(data.replace('0x', ''))
        return keccak(data)
    except Exception:
        return b'\x00' * 32

def generate_random_keys(n):
    """Gera n chaves aleatórias."""
    return [int.from_bytes(np.random.bytes(32), 'big') % (2**256 - 2**32 - 977) + 1
            for _ in range(n)]

def generate_addresses_cpu(keys):
    """Gera endereços Bitcoin na CPU usando geração em lote."""
    batch_size = len(keys)
    addresses = np.zeros((batch_size, 20), dtype=np.uint8)

    for i, key in enumerate(keys):
        try:
            key_hex = f"{key:064x}"
            pk_bytes = bytes.fromhex(key_hex)
            public_key = PublicKey.from_valid_secret(pk_bytes).format(compressed=False)[1:]
            hash_bytes = custom_keccak(public_key)[-20:]
            addresses[i] = np.frombuffer(hash_bytes, dtype=np.uint8)
        except Exception:
            pass

    return addresses

def test_cpu_to_gpu_transfer(sizes):
    """Testa a transferência de dados CPU para GPU para vários tamanhos."""
    if not HAS_CUPY:
        print("❌ CuPy não disponível para teste de transferência")
        return

    results = []

    print("🔄 Testando transferência CPU→GPU para diferentes tamanhos de lote")
    print("=" * 60)
    print(f"{'Tamanho':>10} | {'Tempo (ms)':>12} | {'Taxa (GB/s)':>12}")
    print("-" * 60)

    for size in sizes:
        # Gerar dados na CPU
        cpu_data = np.random.randint(0, 256, size=(size, 20), dtype=np.uint8)

        # Medir tempo de transferência para GPU
        start_time = time.time()
        gpu_data = cp.asarray(cpu_data)
        cp.cuda.stream.get_current_stream().synchronize()
        elapsed = (time.time() - start_time) * 1000  # em ms

        # Calcular taxa de transferência
        bytes_transferred = cpu_data.nbytes
        transfer_rate = bytes_transferred / (elapsed / 1000) / (1024**3)  # em GB/s

        results.append((size, elapsed, transfer_rate))
        print(f"{size:>10} | {elapsed:12.2f} | {transfer_rate:12.2f}")

        # Liberar memória
        del gpu_data
        cp.get_default_memory_pool().free_all_blocks()

    return results

def test_batch_processing(sizes):
    """Testa o processamento em lote para diferentes tamanhos."""
    if not HAS_CUPY:
        print("❌ CuPy não disponível para teste de processamento")
        return

    results = []

    print("\n🔄 Testando processamento em lote para diferentes tamanhos")
    print("=" * 80)
    print(f"{'Tamanho':>10} | {'Geração CPU (ms)':>16} | {'Verificação GPU (ms)':>18} | {'Total (ms)':>12} | {'Taxa (Mend/s)':>12}")
    print("-" * 80)

    # Gerar alguns targets fictícios
    n_targets = 10
    targets = np.random.randint(0, 256, size=(n_targets, 20), dtype=np.uint8)
    targets_gpu = cp.asarray(targets)

    for size in sizes:
        # PARTE 1: Gerar chaves e endereços na CPU
        keys = generate_random_keys(size)

        start_time = time.time()
        addresses = generate_addresses_cpu(keys)
        cpu_time = (time.time() - start_time) * 1000  # em ms

        # PARTE 2: Verificar correspondências na GPU
        start_time = time.time()

        # Transferir para GPU
        addresses_gpu = cp.asarray(addresses)

        # Verificar correspondências
        match_any = cp.zeros(size, dtype=cp.bool_)

        for t in range(n_targets):
            target = targets_gpu[t]
            matches = cp.all(addresses_gpu == target, axis=1)
            match_any = cp.logical_or(match_any, matches)

        # Transferir resultados de volta
        matches_cpu = match_any.get()

        gpu_time = (time.time() - start_time) * 1000  # em ms

        # Estatísticas
        total_time = cpu_time + gpu_time
        throughput = size / (total_time / 1000) / 1e6  # milhões de endereços/segundo

        results.append((size, cpu_time, gpu_time, total_time, throughput))
        print(f"{size:>10} | {cpu_time:16.2f} | {gpu_time:18.2f} | {total_time:12.2f} | {throughput:12.2f}")

        # Limpar memória
        del addresses_gpu, match_any
        cp.get_default_memory_pool().free_all_blocks()

    return results

def show_recommendations(transfer_results, processing_results):
    """Mostra recomendações baseadas nos resultados dos testes."""
    if not transfer_results or not processing_results:
        return

    print("\n" + "="*60)
    print("🔍 ANÁLISE E RECOMENDAÇÕES")
    print("="*60)

    # Encontrar melhor tamanho de lote para transferência
    best_transfer = max(transfer_results, key=lambda x: x[2])
    print(f"✅ Melhor tamanho para transferência: {best_transfer[0]}")
    print(f"   Taxa: {best_transfer[2]:.2f} GB/s")

    # Encontrar melhor tamanho de lote para processamento
    best_processing = max(processing_results, key=lambda x: x[4])
    print(f"\n✅ Melhor tamanho para processamento: {best_processing[0]}")
    print(f"   Throughput: {best_processing[4]:.2f} Mend/s")

    # Análise de gargalos
    print("\n📊 Análise de gargalos:")

    # Verificar se CPU é gargalo
    cpu_times = [r[1] for r in processing_results]
    gpu_times = [r[2] for r in processing_results]

    cpu_avg_ratio = sum(cpu_times) / sum(gpu_times) if sum(gpu_times) > 0 else float('inf')

    if cpu_avg_ratio > 2.0:
        print("⚠️ A geração de endereços na CPU é um gargalo significativo")
        print(f"   CPU leva {cpu_avg_ratio:.1f}x mais tempo que GPU")
        print("   Recomendação: Implementar paralelismo na geração de endereços")

    # Verificar eficiência de transferência
    sizes = [r[0] for r in transfer_results]
    transfer_rates = [r[2] for r in transfer_results]

    if max(transfer_rates) / min(transfer_rates) > 3.0:
        print("\n⚠️ Grande variação na eficiência de transferência")
        print("   Recomendação: Preferir tamanhos de lote maiores para transfers")

    # Recomendação final baseada nos resultados
    print("\n🚀 RECOMENDAÇÕES FINAIS:")

    # Escolher tamanho de lote balanceado
    recommended_batch = best_processing[0]

    # Tamanho para sub-batch baseado no tamanho de memória
    try:
        import cupy as cp
        free_mem, total_mem = cp.cuda.runtime.memGetInfo()
        available_bytes = free_mem * 0.8  # 80% da memória livre

        # Tamanho aproximado por registro (endereço + estruturas auxiliares)
        bytes_per_record = 100

        max_elements = int(available_bytes / bytes_per_record)

        # Arredondar para potência de 2 mais próxima
        max_power_of_2 = 2**int(np.log2(max_elements))

        # Limitar o sub-batch a um máximo razoável
        max_subbatch = min(max_power_of_2, 2**26)  # Máximo de 64M

        recommended_subbatch = max_subbatch
    except:
        # Valor conservador se falhar
        recommended_subbatch = 2**24

    print(f"1. BATCH_SIZE = {recommended_batch}")
    print(f"2. SUBBATCH_SIZE = {recommended_subbatch} ({recommended_subbatch:,})")
    print("3. Implementar paralelização na geração de endereços")

    # Código para fácil cópia e cola
    print("\nCódigo para atualizar no seu script:")
    print("-" * 40)
    print(f"BATCH_SIZE = {recommended_batch} if HAS_CUDA else 8192")
    print(f"SUBBATCH_SIZE = {recommended_subbatch} if HAS_CUDA else 2**20")

def main():
    # Versão modificada para funcionar tanto no Colab quanto na linha de comando

    # Detectar ambiente de execução e definir parâmetros apropriados
    test_mode = "standard"  # default

    if IS_NOTEBOOK or IS_COLAB:
        # Se estamos em um notebook/Colab, ignoramos os argumentos da linha de comando
        # e oferecemos uma interface baseada em variáveis
        print("💡 Executando no ambiente Notebook/Colab - ignorando argumentos de linha de comando")

        # Opcionalmente, podemos permitir ao usuário escolher o modo através de uma variável
        try:
            # Verifica se estamos no Colab e oferecemos widgets interativos
            if IS_COLAB:
                from google.colab import output
                from IPython.display import display, HTML

                print("\n🎮 Selecione o modo de teste:")
                print("1. Mínimo (rápido, poucos tamanhos)")
                print("2. Padrão (equilíbrio entre tempo e precisão)")
                print("3. Completo (mais preciso, leva mais tempo)")

                choice = input("Escolha [1-3] (padrão: 2): ").strip()

                if choice == "1":
                    test_mode = "minimal"
                elif choice == "3":
                    test_mode = "full"
                else:
                    test_mode = "standard"

        except (ImportError, Exception) as e:
            print(f"⚠️ Não foi possível exibir widgets interativos: {e}")
            print("⚠️ Usando modo de teste padrão")
            test_mode = "standard"
    else:
        # Ambiente de linha de comando normal - podemos usar argparse
        import argparse
        parser = argparse.ArgumentParser(description='Teste de tamanhos de batch para GPU')
        parser.add_argument('--minimal', action='store_true', help='Executar teste mínimo (mais rápido)')
        parser.add_argument('--full', action='store_true', help='Executar teste completo (mais demorado)')
        args = parser.parse_args()

        if args.minimal:
            test_mode = "minimal"
        elif args.full:
            test_mode = "full"

    print("=" * 60)
    print("🚀 OTIMIZADOR DE BATCH SIZE PARA MINERADOR BITCOIN 🚀")
    print("=" * 60)

    # Verificação de hardware
    print("\n🔍 Verificando hardware disponível...")
    if IS_COLAB:
        print("✅ Ambiente Google Colab detectado")

        # Verificar GPU no Colab
        try:
            gpu_info = subprocess.run("nvidia-smi", shell=True, stdout=subprocess.PIPE).stdout.decode('utf-8')
            print("\n📊 Informações da GPU:")
            for line in gpu_info.split("\n")[:8]:
                print(f"   {line}")
        except:
            print("❌ nvidia-smi falhou. GPU não disponível?")

    if not HAS_CUPY:
        print("\n❌ CuPy não disponível. Não é possível executar testes na GPU.")
        print("   Instale com: pip install cupy-cuda11x ou cupy-cuda12x dependendo da sua versão CUDA")
        return

    # Definir tamanhos de teste com base no modo selecionado
    if test_mode == "minimal":
        print("\n🔍 Modo: Teste Mínimo (Rápido)")
        batch_sizes = [1024, 8192, 32768]
    elif test_mode == "full":
        print("\n🔍 Modo: Teste Completo (Detalhado)")
        batch_sizes = [512, 1024, 2048, 4096, 8192, 16384, 32768, 65536]
        # Verificar se há memória suficiente para testar tamanhos maiores
        if HAS_CUPY:
            try:
                free_mem, _ = cp.cuda.runtime.memGetInfo()
                if free_mem > 6 * (1024**3):  # Mais de 6GB livre
                    batch_sizes.append(131072)  # Adicionar teste de 128K
                    print("⚠️ Executando teste de tamanho grande (128K)")
            except:
                pass
    else:
        print("\n🔍 Modo: Teste Padrão")
        batch_sizes = [1024, 4096, 8192, 16384, 32768, 65536]

    print(f"\n🔢 Testando tamanhos de batch: {batch_sizes}")

    # Testar transferência CPU→GPU
    transfer_results = test_cpu_to_gpu_transfer(batch_sizes)

    # Testar processamento de lotes
    processing_results = test_batch_processing(batch_sizes)

    # Mostrar recomendações
    show_recommendations(transfer_results, processing_results)

if __name__ == "__main__":
    main()


In [ ]:
"""
Ferramenta para otimizar o desempenho GPU para o minerador Bitcoin
Resolve o problema de baixo desempenho GPU comparado à CPU
"""

import time
import numpy as np
import sys
import subprocess

# Tentar importar CuPy, PyTorch e TensorFlow para testes
GPU_LIBS = {}

try:
    import cupy as cp
    GPU_LIBS["cupy"] = True
    print("✅ CuPy disponível")
except ImportError:
    GPU_LIBS["cupy"] = False
    print("❌ CuPy não disponível")

try:
    import torch
    GPU_LIBS["torch"] = torch.cuda.is_available()
    print(f"✅ PyTorch disponível, GPU: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   Device: {torch.cuda.get_device_name(0)}")
except ImportError:
    GPU_LIBS["torch"] = False
    print("❌ PyTorch não disponível")

try:
    import tensorflow as tf
    GPU_LIBS["tensorflow"] = len(tf.config.list_physical_devices('GPU')) > 0
    print(f"✅ TensorFlow disponível, GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")
except ImportError:
    GPU_LIBS["tensorflow"] = False
    print("❌ TensorFlow não disponível")

def test_gpu_memory_transfer():
    """Testa a velocidade de transferência de dados entre CPU e GPU"""
    if not GPU_LIBS["cupy"]:
        print("❌ CuPy não disponível para teste de transferência")
        return

    print("\n🔍 Testando velocidade de transferência CPU <-> GPU...")

    sizes = [
        (1024*1024, "1MB"),
        (10*1024*1024, "10MB"),
        (100*1024*1024, "100MB"),
        (500*1024*1024, "500MB"),
    ]

    for size_bytes, size_name in sizes:
        # Criar array na CPU
        cpu_array = np.ones(size_bytes // 4, dtype=np.float32)

        # Teste de upload (CPU -> GPU)
        start = time.time()
        gpu_array = cp.asarray(cpu_array)
        cp.cuda.stream.get_current_stream().synchronize()
        upload_time = time.time() - start
        upload_speed = size_bytes / upload_time / (1024**3)  # GB/s

        # Teste de download (GPU -> CPU)
        start = time.time()
        cpu_result = gpu_array.get()
        download_time = time.time() - start
        download_speed = size_bytes / download_time / (1024**3)  # GB/s

        print(f"📦 Tamanho: {size_name}")
        print(f"   Upload (CPU→GPU): {upload_time*1000:.1f}ms ({upload_speed:.2f} GB/s)")
        print(f"   Download (GPU→CPU): {download_time*1000:.1f}ms ({download_speed:.2f} GB/s)")

        # Limpar memória
        del gpu_array
        cp.get_default_memory_pool().free_all_blocks()

def optimize_batch_size():
    """Encontra o tamanho de lote ideal para processamento na GPU"""
    if not GPU_LIBS["cupy"]:
        print("❌ CuPy não disponível para teste de tamanho de lote")
        return

    print("\n🔍 Encontrando tamanho de lote ideal para GPU...")

    # Definir diferentes tamanhos de lote para teste
    batch_sizes = [512, 1024, 2048, 4096, 8192, 16384, 32768, 65536]
    vector_size = 20  # Tamanho do vetor para cada elemento (20 bytes para hash160)
    target_count = 10  # Número de targets para comparar

    best_batch_size = None
    best_throughput = 0

    for batch_size in batch_sizes:
        print(f"\n📊 Testando batch_size={batch_size}...")

        # Criar dados de teste
        addresses_cpu = np.random.randint(0, 256, size=(batch_size, vector_size), dtype=np.uint8)
        targets_cpu = np.random.randint(0, 256, size=(target_count, vector_size), dtype=np.uint8)

        # Transferir para GPU
        start_time = time.time()
        addresses_gpu = cp.asarray(addresses_cpu)
        targets_gpu = cp.asarray(targets_cpu)

        # Simular operações de verificação (comparação com cada target)
        results = cp.zeros(batch_size, dtype=cp.bool_)

        for t in range(target_count):
            target = targets_gpu[t]

            # Comparar todos os vetores com este target
            match_all = cp.ones(batch_size, dtype=cp.bool_)
            for b in range(vector_size):
                match_this_byte = addresses_gpu[:, b] == target[b]
                match_all = match_all & match_this_byte

            # Adicionar aos resultados
            results = results | match_all

        # Obter resultados de volta
        matches = results.get()

        total_time = time.time() - start_time
        throughput = batch_size / total_time

        print(f"   Tempo: {total_time:.4f}s")
        print(f"   Throughput: {throughput:.0f} elementos/s")
        print(f"   Throughput: {throughput/1e6:.2f} Melementos/s")

        if throughput > best_throughput:
            best_throughput = throughput
            best_batch_size = batch_size

        # Limpar memória GPU
        del addresses_gpu, targets_gpu, results
        cp.get_default_memory_pool().free_all_blocks()

    # Resumo final
    print(f"\n✅ RESULTADO FINAL:")
    print(f"   Melhor tamanho de lote: {best_batch_size}")
    print(f"   Throughput máximo: {best_throughput/1e6:.2f} Melementos/s")
    print(f"   ➡️ Recomendação: Definir BATCH_SIZE = {best_batch_size} no seu script")

    return best_batch_size

def optimize_memory_usage():
    """Otimiza o uso de memória GPU para melhor desempenho"""
    if not GPU_LIBS["cupy"]:
        return

    print("\n🔍 Otimizando uso de memória GPU...")

    try:
        # Obter informações de memória total e disponível
        mem_info = cp.cuda.runtime.memGetInfo()
        mem_free = mem_info[0]
        mem_total = mem_info[1]

        print(f"   Memória total: {mem_total / (1024**2):.0f} MB")
        print(f"   Memória disponível: {mem_free / (1024**2):.0f} MB")

        # Calcular fração segura para uso
        safe_fraction = 0.8  # Usa 80% da memória disponível
        safe_mem = int(mem_free * safe_fraction)

        # Estimar quantos elementos podem ser processados com essa memória
        # Assumindo que cada elemento usa ~100 bytes na GPU (endereço + dados complementares)
        bytes_per_element = 100
        max_elements = safe_mem // bytes_per_element

        print(f"   Memória segura para uso: {safe_mem / (1024**2):.0f} MB")
        print(f"   Elementos estimados: {max_elements:,}")

        # Calcular batch size recomendado (arredondar para potência de 2 inferior)
        batch_size = 2**int(np.log2(max_elements))
        batch_size = min(batch_size, 65536)  # Limitar ao máximo razoável

        print(f"   ➡️ Batch size recomendado: {batch_size}")

        # Sugestão para sub-batch size (maior para GPU)
        subbatch_size = 2**24  # 16M chaves por sub-lote
        if mem_total < 8 * (1024**3):  # Menos de 8GB de VRAM
            subbatch_size = 2**22  # Reduz para 4M em GPUs com menos memória

        print(f"   ➡️ Sub-batch size recomendado: {subbatch_size:,}")

        return batch_size, subbatch_size
    except Exception as e:
        print(f"❌ Erro ao calcular uso de memória: {e}")
        return None, None

def test_cpu_vs_gpu():
    """Compara desempenho da CPU vs GPU para operações específicas do minerador"""
    print("\n🏆 COMPARANDO DESEMPENHO CPU vs GPU")

    # Definir tamanho do teste
    sample_size = 50000  # 50K elementos

    # Gerar dados aleatórios para teste
    data_np = np.random.randint(0, 256, size=(sample_size, 20), dtype=np.uint8)
    targets_np = np.random.randint(0, 256, size=(10, 20), dtype=np.uint8)

    # Teste 1: Comparação na CPU usando NumPy
    print("\n📊 Teste de comparação (CPU/NumPy):")
    start_time = time.time()
    matches_cpu = np.zeros(sample_size, dtype=bool)

    for i in range(sample_size):
        for t in range(10):
            if np.array_equal(data_np[i], targets_np[t]):
                matches_cpu[i] = True
                break

    cpu_time = time.time() - start_time

    print(f"   Tempo CPU: {cpu_time:.4f}s")
    print(f"   Throughput: {sample_size / cpu_time:.0f} elem/s")

    # Teste 2: Comparação na GPU usando CuPy
    if GPU_LIBS["cupy"]:
        print("\n📊 Teste de comparação (GPU/CuPy):")
        start_time = time.time()

        # Transferir dados para GPU
        data_cp = cp.asarray(data_np)
        targets_cp = cp.asarray(targets_np)

        # Realizar comparação
        matches_gpu = cp.zeros(sample_size, dtype=cp.bool_)

        for t in range(10):
            target = targets_cp[t]
            match_all = cp.all(data_cp == target, axis=1)
            matches_gpu = matches_gpu | match_all

        # Forçar sincronização e copiar resultados
        cp.cuda.stream.get_current_stream().synchronize()
        matches_gpu_np = matches_gpu.get()

        gpu_time = time.time() - start_time

        print(f"   Tempo GPU: {gpu_time:.4f}s")
        print(f"   Throughput: {sample_size / gpu_time:.0f} elem/s")
        print(f"   Speedup GPU/CPU: {cpu_time / gpu_time:.2f}x")

        # Verificar resultados
        matches = np.sum(matches_cpu == matches_gpu_np)
        accuracy = matches / sample_size * 100
        print(f"   Resultados equivalentes: {accuracy:.2f}%")

        # Limpar memória GPU
        del data_cp, targets_cp, matches_gpu
        cp.get_default_memory_pool().free_all_blocks()

def main():
    print("=" * 60)
    print("🚀 OTIMIZADOR DE DESEMPENHO GPU PARA MINERADOR BITCOIN 🚀")
    print("=" * 60)

    # Verificar se GPU está disponível
    if not any(GPU_LIBS.values()):
        print("❌ Nenhuma biblioteca GPU (CuPy, PyTorch, TensorFlow) disponível.")
        print("   Instale pelo menos uma dessas bibliotecas antes de usar este otimizador.")
        return

    # Testar velocidade de transferência entre CPU e GPU
    test_gpu_memory_transfer()

    # Otimizar tamanho de lote para GPU
    best_batch_size = optimize_batch_size()

    # Otimizar uso de memória
    mem_batch_size, subbatch_size = optimize_memory_usage()

    # Comparar CPU vs GPU
    test_cpu_vs_gpu()

    print("\n" + "=" * 60)
    print("✅ RECOMENDAÇÕES FINAIS")
    print("=" * 60)

    # Determinar tamanho final recomendado
    batch_size = best_batch_size if best_batch_size else mem_batch_size
    if not batch_size:
        batch_size = 8192  # valor padrão conservador

    print(f"1. Defina BATCH_SIZE = {batch_size}")
    print(f"2. Defina SUBBATCH_SIZE = {subbatch_size or 2**23}")
    print(f"3. Para melhor desempenho com CuPy, minimize transferências entre CPU e GPU")
    print(f"4. Pré-aloque buffers na GPU para reduzir fragmentação de memória")
    print(f"5. Processe os lotes em blocos para operações na GPU")
    print("=" * 60)

if __name__ == "__main__":
    main()


In [ ]:
"""
Script para testar a velocidade de processamento de diferentes métodos
e identificar gargalos de desempenho
"""
import time
import numpy as np
from coincurve import PublicKey
from eth_utils import keccak
import base58
import hashlib

# Tente importar bibliotecas GPU
try:
    import cupy as cp
    HAS_CUPY = True
    print("✅ CuPy disponível")
except ImportError:
    HAS_CUPY = False
    print("❌ CuPy não disponível")

try:
    import torch
    HAS_TORCH = torch.cuda.is_available()
    print(f"✅ PyTorch disponível, GPU: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   Device: {torch.cuda.get_device_name(0)}")
except ImportError:
    HAS_TORCH = False
    print("❌ PyTorch não disponível")

try:
    import tensorflow as tf
    HAS_TF = len(tf.config.list_physical_devices('GPU')) > 0
    print(f"✅ TensorFlow disponível, GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")
except ImportError:
    HAS_TF = False
    print("❌ TensorFlow não disponível")

# Funções de hash e endereçamento
def custom_keccak(data):
    """Calcula Keccak-256 com tratamento de erros."""
    try:
        if isinstance(data, str):
            data = bytes.fromhex(data.replace('0x', ''))
        return keccak(data)
    except Exception:
        return b'\x00' * 32

def bitcoin_hash160(data):
    """Calcula hash160 (SHA-256 seguido de RIPEMD-160)"""
    sha = hashlib.sha256(data).digest()
    ripemd160 = hashlib.new('ripemd160')
    ripemd160.update(sha)
    return ripemd160.digest()

def test_address_generation(n_keys=1000):
    """Testa a velocidade da geração de endereços Bitcoin."""
    print(f"\n🔄 Testando geração de {n_keys} endereços Bitcoin...")

    # Gerar chaves privadas aleatórias
    print("Gerando chaves privadas...")
    keys = [int.from_bytes(np.random.bytes(32), 'big') % (2**256 - 2**32 - 977) + 1 for _ in range(n_keys)]

    # Método 1: Keccak-256 (ethereum)
    print("\nMétodo 1: Keccak-256 (Ethereum)")
    start_time = time.time()
    addresses_keccak = []
    for key in keys:
        try:
            private_key_hex = f"{key:064x}"
            pk_bytes = bytes.fromhex(private_key_hex)
            public_key = PublicKey.from_valid_secret(pk_bytes).format(compressed=False)[1:]
            hash_bytes = custom_keccak(public_key)[-20:]
            addresses_keccak.append(hash_bytes)
        except Exception:
            addresses_keccak.append(b'\x00' * 20)
    elapsed = time.time() - start_time
    print(f"✅ Tempo: {elapsed:.3f}s ({n_keys/elapsed:.1f} chaves/s)")

    # Método 2: SHA-256 + RIPEMD-160 (bitcoin)
    print("\nMétodo 2: SHA-256 + RIPEMD-160 (Bitcoin)")
    start_time = time.time()
    addresses_bitcoin = []
    for key in keys:
        try:
            private_key_hex = f"{key:064x}"
            pk_bytes = bytes.fromhex(private_key_hex)
            public_key = PublicKey.from_valid_secret(pk_bytes).format(compressed=False)[1:]
            hash_bytes = bitcoin_hash160(public_key)
            addresses_bitcoin.append(hash_bytes)
        except Exception:
            addresses_bitcoin.append(b'\x00' * 20)
    elapsed = time.time() - start_time
    print(f"✅ Tempo: {elapsed:.3f}s ({n_keys/elapsed:.1f} chaves/s)")

    # Comparar resultados
    matches = 0
    for i in range(len(addresses_keccak)):
        if addresses_keccak[i] == addresses_bitcoin[i]:
            matches += 1
    match_percent = (matches / len(addresses_keccak)) * 100
    print(f"📊 Correspondência entre métodos: {match_percent:.2f}%")

    if match_percent < 100:
        print("⚠️ AVISO: Os métodos produzem resultados diferentes!")
        print("   Isso pode causar incompatibilidade com endereços Bitcoin reais.")

def test_batch_processing():
    """Testa a velocidade de processamento em lotes."""
    print("\n🔄 Testando velocidade de processamento em lote...")

    # Preparar dados de teste
    n_addresses = 10000
    n_targets = 10

    # Gerar endereços aleatórios
    addresses = np.random.randint(0, 256, size=(n_addresses, 20), dtype=np.uint8)
    # Garantir que pelo menos 5 endereços correspondem a targets
    targets = np.random.randint(0, 256, size=(n_targets, 20), dtype=np.uint8)
    for i in range(5):
        addresses[i] = targets[i % n_targets]

    # Método 1: Comparação com NumPy
    print("\nMétodo 1: NumPy (CPU)")
    start_time = time.time()
    matches_numpy = np.zeros(n_addresses, dtype=bool)
    for i in range(n_addresses):
        for t in range(n_targets):
            if np.array_equal(addresses[i], targets[t]):
                matches_numpy[i] = True
                break
    numpy_time = time.time() - start_time
    print(f"✅ Tempo: {numpy_time:.3f}s ({n_addresses/numpy_time:.1f} endereços/s)")
    print(f"   Matches: {np.sum(matches_numpy)}")

    # Método 2: CuPy (se disponível)
    if HAS_CUPY:
        print("\nMétodo 2: CuPy (GPU)")
        start_time = time.time()

        # Transferir para GPU
        addresses_gpu = cp.asarray(addresses)
        targets_gpu = cp.asarray(targets)
        matches_cupy = cp.zeros(n_addresses, dtype=bool)

        # Verificar correspondências
        for t_idx in range(n_targets):
            target = targets_gpu[t_idx]
            # Comparar cada endereço com este target
            equal_bytes = cp.all(addresses_gpu == target, axis=1)
            # Atualizar resultados
            matches_cupy = cp.logical_or(matches_cupy, equal_bytes)

        # Transferir resultados de volta para CPU
        matches_cupy_cpu = matches_cupy.get()

        cupy_time = time.time() - start_time
        print(f"✅ Tempo: {cupy_time:.3f}s ({n_addresses/cupy_time:.1f} endereços/s)")
        print(f"   Matches: {np.sum(matches_cupy_cpu)}")
        print(f"   Speedup vs CPU: {numpy_time/cupy_time:.1f}x")

if __name__ == "__main__":
    print("=" * 60)
    print("🚀 TESTE DE DESEMPENHO BITCOIN MINER 🚀")
    print("=" * 60)

    # Teste de geração de endereços
    test_address_generation(5000)

    # Teste de processamento em lote
    test_batch_processing()


In [ ]:
"""
Script para validar a correta conversão de chaves privadas para endereços Bitcoin
"""
import base58
import numpy as np
from coincurve import PublicKey
from eth_utils import keccak
import hashlib
import subprocess
import sys

# Instalar bibliotecas necessárias se não estiverem disponíveis
try:
    from Crypto.Hash import RIPEMD160
    HAS_PYCRYPTO = True
    print("✅ Usando Crypto.Hash.RIPEMD160")
except ImportError:
    HAS_PYCRYPTO = False
    print("⚠️ Crypto.Hash.RIPEMD160 não disponível, instalando pycryptodome...")
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "pycryptodome"], check=True)
        from Crypto.Hash import RIPEMD160
        HAS_PYCRYPTO = True
        print("✅ pycryptodome instalado com sucesso")
    except:
        print("❌ Não foi possível instalar pycryptodome")
        HAS_PYCRYPTO = False

def custom_keccak(data):
    """Calcula Keccak-256 com tratamento de erros."""
    try:
        if isinstance(data, str):
            data = bytes.fromhex(data.replace('0x', ''))
        return keccak(data)
    except Exception:
        return b'\x00' * 32

def sha256(data):
    """Calcula SHA-256"""
    return hashlib.sha256(data).digest()

def ripemd160(data):
    """
    Calcula RIPEMD-160 usando a biblioteca Crypto.Hash quando disponível
    ou um método alternativo quando não está disponível
    """
    if HAS_PYCRYPTO:
        # Usando pycryptodome
        h = RIPEMD160.new()
        h.update(data)
        return h.digest()
    else:
        # Implementação alternativa usando outro algoritmo
        # (isso é apenas um fallback, não use em produção)
        print("⚠️ RIPEMD160 não disponível, usando SHA1 como fallback (NÃO SEGURO)")
        return hashlib.sha1(data).digest()

def bitcoin_hash160(public_key):
    """Implementação correta do hash160 usado no Bitcoin (SHA256 seguido de RIPEMD160)"""
    sha = sha256(public_key)
    ripe = ripemd160(sha)
    return ripe

def bitcoin_address_from_private_key(private_key_int):
    """
    Converte uma chave privada (inteiro) para um endereço Bitcoin.
    Retorna o endereço Base58Check e o hash160 como array.
    """
    try:
        # Converter para bytes
        private_key_hex = f"{private_key_int:064x}"
        private_key_bytes = bytes.fromhex(private_key_hex)

        # Gerar chave pública
        public_key = PublicKey.from_valid_secret(private_key_bytes).format(compressed=False)[1:]

        # Opção 1: Hash usando Keccak (eth_utils)
        hash_keccak = custom_keccak(public_key)[-20:]

        # Opção 2: Hash usando SHA-256 + RIPEMD-160 (método Bitcoin tradicional)
        hash_bitcoin = bitcoin_hash160(public_key)

        # Comparar os dois métodos de hash
        if hash_keccak != hash_bitcoin:
            print(f"⚠️ Discrepância nos métodos de hash para chave {private_key_hex[:8]}...")
            print(f"   Keccak: {hash_keccak.hex()}")
            print(f"   Bitcoin: {hash_bitcoin.hex()}")

        # Usar o método Bitcoin (SHA-256 + RIPEMD-160)
        hash160 = hash_bitcoin

        # Adicionar byte de versão (0x00 para Bitcoin mainnet)
        extended = b'\x00' + hash160

        # Calcular checksum (4 primeiros bytes do SHA-256 duplo)
        checksum = sha256(sha256(extended))[:4]

        # Juntar tudo
        address_bytes = extended + checksum

        # Codificar em Base58
        address = base58.b58encode(address_bytes).decode('ascii')

        return address, np.frombuffer(hash160, dtype=np.uint8)
    except Exception as e:
        print(f"❌ Erro ao converter chave privada: {e}")
        return None, None

def test_conversion():
    """Testa a conversão com chaves privadas e endereços conhecidos."""
    # Alguns exemplos conhecidos de pares chave privada -> endereço Bitcoin
    test_cases = [
        # Formato: (chave privada em hex, endereço esperado)
        ("1", "1EHNa6Q4Jz2uvNExL497mE43ikXhwF6kZm"),
        ("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364140", "1LdHYK73XVvknPTLoTVW7hm3H6LWNdZQKL"),
        ("0000000000000000000000000000000000000000000000000000000000000001", "1BgGZ9tcN4rm9KBzDn7KprQz87SZ26SAMH")
    ]

    print("🔍 Validando conversão de chaves privadas para endereços Bitcoin...")

    for i, (priv_key_hex, expected_address) in enumerate(test_cases):
        priv_key_int = int(priv_key_hex, 16)
        generated_address, hash160 = bitcoin_address_from_private_key(priv_key_int)

        if generated_address == expected_address:
            print(f"✅ Teste {i+1}: Endereço correto: {generated_address}")
        else:
            print(f"❌ Teste {i+1}: Erro! Esperado: {expected_address}, obtido: {generated_address}")

    # Teste especial para o endereço adicional
    golden_address = "1MVDYgVaSN6iKKEsbzRUAYFrYJadLYZvvZ"

    # Decodificar o endereço para obter o hash160
    decoded = base58.b58decode(golden_address)
    golden_hash = decoded[1:-4]  # Remove byte de versão e checksum

    print(f"\n🌟 Golden Address: {golden_address}")
    print(f"   Hash160: {golden_hash.hex()}")

    # Para validação, podemos verificar se o endereço reconstruído corresponde ao original
    extended = b'\x00' + golden_hash
    checksum = sha256(sha256(extended))[:4]
    address_bytes = extended + checksum
    reconstructed = base58.b58encode(address_bytes).decode('ascii')

    if reconstructed == golden_address:
        print(f"✅ Validação do Golden Address bem-sucedida")
    else:
        print(f"❌ Validação do Golden Address falhou")

    print("\n🔄 O minerador está usando o algoritmo de hash correto para Bitcoin? Verificando...")
    # Verificar se estamos usando o algoritmo correto para Bitcoin
    if "eth_utils" in globals():
        print("⚠️ O código está usando 'eth_utils.keccak' em vez de SHA-256+RIPEMD-160.")
        print("   Isso pode causar incompatibilidade com endereços Bitcoin.")
    else:
        print("✅ O código está usando SHA-256+RIPEMD-160, que é o algoritmo correto para Bitcoin.")

if __name__ == "__main__":
    print("=" * 60)
    print("🧪 TESTE DE VALIDAÇÃO DE CHAVES BITCOIN 🧪")
    print("=" * 60)
    test_conversion()
    print("=" * 60)


In [ ]:
"""
Script para otimizar o ambiente de mineração com base nos resultados do teste de batch size
Configura o sistema para máximo desempenho de acesso à GPU
"""

import os
import sys
import subprocess
import time
import json
import multiprocessing

def check_environment():
    """Verifica o ambiente de execução e mostra informações relevantes."""
    print("=" * 60)
    print("🔍 VERIFICANDO AMBIENTE DE MINERAÇÃO")
    print("=" * 60)

    # Verificar se estamos no Google Colab
    is_colab = 'google.colab' in sys.modules
    if is_colab:
        print("✅ Ambiente Google Colab detectado")

        # Verificar informações da GPU
        try:
            gpu_info = subprocess.run("nvidia-smi", shell=True, stdout=subprocess.PIPE).stdout.decode('utf-8')
            print("\n📊 Informações da GPU:")
            for line in gpu_info.split('\n')[:10]:
                print(f"   {line}")
        except:
            print("❌ GPU não detectada ou nvidia-smi falhou")
    else:
        print("ℹ️ Executando fora do Google Colab")

    # Verificar CPUs disponíveis
    cpu_count = multiprocessing.cpu_count()
    print(f"\n📊 CPUs disponíveis: {cpu_count}")

    # Verificar memória do sistema
    try:
        if is_colab:
            # Usar comando para obter memória no Linux
            mem_info = subprocess.run("free -m", shell=True, stdout=subprocess.PIPE).stdout.decode('utf-8')
            print("\n📊 Informações de memória:")
            for line in mem_info.split('\n')[:3]:
                print(f"   {line}")
        else:
            # Método mais genérico
            import psutil
            vm = psutil.virtual_memory()
            print(f"\n📊 Memória sistema: {vm.total / (1024**3):.1f} GB total, {vm.available / (1024**3):.1f} GB disponível")
    except:
        print("⚠️ Não foi possível obter informações de memória")

    return is_colab

def load_batch_test_results():
    """Carrega resultados do teste de batch size se disponível, ou usa valores padrão."""
    try:
        with open("batch_test_results.json", "r") as f:
            results = json.load(f)
            print("✅ Carregados resultados do teste de batch size anterior")
            return results
    except:
        # Valores padrão caso não tenhamos um arquivo de resultados
        print("⚠️ Arquivo de resultados não encontrado, usando valores padrão")
        return {
            "batch_size": 32768,
            "subbatch_size": 67108864,
            "cpu_is_bottleneck": True,
            "recommended_workers": multiprocessing.cpu_count() - 1
        }

def update_runtime_config():
    """Atualiza configurações do ambiente de execução para melhor desempenho."""
    print("\n" + "=" * 60)
    print("🔧 OTIMIZANDO CONFIGURAÇÕES DE RUNTIME")
    print("=" * 60)

    # Carregar configurações do núcleo do sistema (Linux)
    try:
        # Configurar para melhor desempenho multithreading
        if sys.platform.startswith('linux'):
            # Desativar preemption para melhor desempenho de CPU
            subprocess.run("echo 0 | sudo tee /proc/sys/kernel/sched_rt_runtime_us",
                          shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            print("✅ Kernel configurado para priorizar tasks de tempo real")

            # Configurar para melhor desempenho de I/O
            subprocess.run("echo 3 | sudo tee /proc/sys/vm/drop_caches",
                          shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            print("✅ Caches de sistema liberados para melhor desempenho")
    except:
        print("⚠️ Não foi possível otimizar configurações do kernel")

    # Configurar Python para melhor desempenho
    try:
        # Ajustar GC para menos interrupções
        import gc
        gc.disable()
        print("✅ Garbage collector desabilitado para melhor desempenho")
    except:
        pass

    # Configurar NumPy para usar múltiplas threads
    try:
        import numpy as np
        np.show_config()
        print("⚠️ Verifique se NumPy está usando MKL para otimização de performance")
    except:
        pass

    return True

def optimize_nvidia_settings():
    """Otimiza configurações específicas da GPU NVIDIA."""
    try:
        # Definir modo de performance para máximo desempenho
        subprocess.run("nvidia-smi -pm 1", shell=True,
                      stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Desativar limitação de potência
        subprocess.run("nvidia-smi -pl 250", shell=True,  # 250W ou ajustar conforme sua GPU
                      stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Configurar para modo de computação
        subprocess.run("nvidia-smi -c 3", shell=True,  # Modo EXCLUSIVE_PROCESS
                      stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        print("✅ GPU NVIDIA otimizada para máximo desempenho")
        return True
    except:
        print("⚠️ Não foi possível otimizar configurações NVIDIA")
        return False

def generate_optimized_config():
    """Gera um arquivo de configuração otimizado para o minerador."""
    # Carregar resultados do teste de batch size
    results = load_batch_test_results()

    # Determinar valores ótimos
    batch_size = results.get("batch_size", 32768)
    subbatch_size = results.get("subbatch_size", 67108864)
    cpu_is_bottleneck = results.get("cpu_is_bottleneck", True)

    # Determinar número de workers com base no número de CPUs
    cpu_count = multiprocessing.cpu_count()
    recommended_workers = max(1, cpu_count - 1)  # Deixar 1 CPU livre para OS

    # Criar configuração
    config = {
        "batch_size": batch_size,
        "subbatch_size": subbatch_size,
        "parallel_workers": recommended_workers,
        "cpu_is_bottleneck": cpu_is_bottleneck,
        "gpu_memory_fraction": 0.9,  # Usar 90% da memória GPU
        "timestamp": time.time()
    }

    # Salvar em arquivo
    try:
        with open("mining_config.json", "w") as f:
            json.dump(config, f, indent=2)

        print("\n✅ Configuração otimizada salva em mining_config.json")
        print(f"   BATCH_SIZE = {batch_size}")
        print(f"   SUBBATCH_SIZE = {subbatch_size}")
        print(f"   PARALLEL_WORKERS = {recommended_workers}")

        # Gerar código para incluir no script
        print("\n📋 Adicione este código ao seu script:")
        print("-" * 60)
        print(f"BATCH_SIZE = {batch_size} if HAS_CUDA else 8192")
        print(f"SUBBATCH_SIZE = {subbatch_size} if HAS_CUDA else 2**20")
        print(f"MAX_PARALLEL_WORKERS = {recommended_workers}")
        print("-" * 60)

        return config
    except Exception as e:
        print(f"❌ Erro ao salvar configuração: {e}")
        return None

def main():
    print("=" * 60)
    print("🚀 OTIMIZADOR DE AMBIENTE DE MINERAÇÃO")
    print("=" * 60)

    # Verificar ambiente
    is_colab = check_environment()

    # Atualizar configurações
    update_runtime_config()

    # Otimizar configurações NVIDIA se possível
    if is_colab:
        optimize_nvidia_settings()

    # Gerar configuração otimizada
    config = generate_optimized_config()

    print("\n" + "=" * 60)
    print("✅ OTIMIZAÇÃO CONCLUÍDA")
    print("=" * 60)

    # Mostrar próximos passos
    print("\n⏭️ Próximos passos:")
    print("1. Execute o minerador com as configurações otimizadas")
    print("2. Monitore o desempenho para verificar se as otimizações foram eficazes")
    print("3. Se necessário, ajuste os parâmetros manualmente")

    if is_colab:
        print("\n⚠️ Lembre-se: No Colab, você pode precisar reiniciar o runtime")
        print("   para que algumas configurações tenham efeito.")

if __name__ == "__main__":
    main()


In [ ]:
"""
Gerador paralelo otimizado de endereços Bitcoin para melhorar o desempenho do minerador
Resolve o gargalo identificado no teste de batch size (geração de endereços na CPU)
"""

import numpy as np
import time
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
from coincurve import PublicKey
import hashlib
import sys

# Verificar se temos suporte a RIPEMD160
try:
    from Crypto.Hash import RIPEMD160
    HAS_PYCRYPTO = True
except ImportError:
    HAS_PYCRYPTO = False
    try:
        # Tentar instalar
        import subprocess
        print("🔄 Instalando pycryptodome para suporte RIPEMD160...")
        subprocess.run([sys.executable, "-m", "pip", "install", "pycryptodome"], check=True)
        from Crypto.Hash import RIPEMD160
        HAS_PYCRYPTO = True
    except:
        HAS_PYCRYPTO = False

def sha256(data):
    """Calcula SHA-256"""
    return hashlib.sha256(data).digest()

def ripemd160(data):
    """Calcula RIPEMD-160 com suporte a diferentes implementações"""
    if HAS_PYCRYPTO:
        h = RIPEMD160.new()
        h.update(data)
        return h.digest()
    else:
        # Usar hashlib ou implementação alternativa
        # Aviso: O hashlib padrão pode não suportar RIPEMD160
        import hashlib
        try:
            h = hashlib.new('ripemd160')
            h.update(data)
            return h.digest()
        except:
            # Último recurso, usar SHA-1 (NÃO RECOMENDADO para produção!)
            print("⚠️ AVISO: Usando SHA-1 como substituto para RIPEMD160 (não seguro)")
            return hashlib.sha1(data).digest()

def hash160(public_key):
    """Implementação padrão Bitcoin: SHA-256 seguido de RIPEMD-160"""
    h = sha256(public_key)
    return ripemd160(h)

def generate_bitcoin_address(private_key):
    """
    Gera um endereço Bitcoin a partir de uma chave privada.

    Args:
        private_key: Chave privada em formato número inteiro

    Returns:
        ndarray: Array NumPy (20 bytes) contendo o hash160 do endereço
    """
    try:
        # Converter para bytes
        key_hex = f"{private_key:064x}"
        key_bytes = bytes.fromhex(key_hex)

        # Gerar chave pública
        public_key = PublicKey.from_valid_secret(key_bytes).format(compressed=False)[1:]

        # Calcular hash160 (SHA-256 + RIPEMD-160)
        hash_bytes = hash160(public_key)

        # Retornar array NumPy
        return np.frombuffer(hash_bytes, dtype=np.uint8)
    except Exception:
        # Retornar zeros em caso de erro
        return np.zeros(20, dtype=np.uint8)

def process_key_chunk(key_chunk, chunk_idx=0):
    """
    Processa um conjunto de chaves em paralelo.

    Args:
        key_chunk: Lista de chaves privadas
        chunk_idx: Índice do chunk (para logging)

    Returns:
        ndarray: Array NumPy (n_keys, 20) contendo os hash160 dos endereços
    """
    n_keys = len(key_chunk)
    addresses = np.zeros((n_keys, 20), dtype=np.uint8)

    start_time = time.time()

    for i, key in enumerate(key_chunk):
        try:
            addresses[i] = generate_bitcoin_address(key)
        except Exception as e:
            # Manter zeros em caso de erro
            if i % 1000 == 0:
                print(f"⚠️ Erro no chunk {chunk_idx}, key {i}: {e}")

    elapsed = time.time() - start_time
    rate = n_keys / elapsed if elapsed > 0 else 0

    if chunk_idx % 10 == 0:  # Reduzir o volume de logs
        print(f"✅ Chunk {chunk_idx}: {n_keys} endereços em {elapsed:.2f}s ({rate:.0f}/s)")

    return addresses

def generate_addresses_parallel(keys, max_workers=None, chunk_size=1000):
    """
    Gera endereços Bitcoin em paralelo para um conjunto de chaves.

    Args:
        keys: Lista de chaves privadas
        max_workers: Número máximo de workers (None = auto, baseado em CPU cores)
        chunk_size: Tamanho do chunk para cada worker processar

    Returns:
        ndarray: Array NumPy (n_keys, 20) contendo os hash160 dos endereços
    """
    if max_workers is None:
        # Usar número de CPUs disponíveis, menos 1 para não travar o sistema
        max_workers = max(1, multiprocessing.cpu_count() - 1)

    n_keys = len(keys)
    print(f"🔄 Gerando {n_keys} endereços com {max_workers} workers em paralelo...")

    # Criar array para resultado final
    all_addresses = np.zeros((n_keys, 20), dtype=np.uint8)

    # Dividir as chaves em chunks
    if chunk_size <= 0:
        chunk_size = max(1, n_keys // (max_workers * 2))

    chunks = []
    for i in range(0, n_keys, chunk_size):
        end = min(i + chunk_size, n_keys)
        chunks.append(keys[i:end])

    n_chunks = len(chunks)
    print(f"📊 Processando {n_chunks} chunks de {chunk_size} chaves cada")

    start_time = time.time()

    try:
        # Usar ProcessPoolExecutor para paralelismo real (multiprocessamento)
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            futures = []

            # Submeter todos os chunks para processamento
            for i, chunk in enumerate(chunks):
                futures.append(executor.submit(process_key_chunk, chunk, i))

            # Coletar resultados na ordem
            for i, future in enumerate(futures):
                try:
                    # Obter resultado do chunk
                    addresses = future.result()

                    # Copiar para o array final
                    start_idx = i * chunk_size
                    end_idx = min(start_idx + len(addresses), n_keys)
                    all_addresses[start_idx:end_idx] = addresses
                except Exception as e:
                    print(f"❌ Erro ao processar chunk {i}: {e}")

    except KeyboardInterrupt:
        print("⚠️ Interrompido pelo usuário")
    except Exception as e:
        print(f"❌ Erro no processamento paralelo: {e}")

        # Tentar abordagem sequencial como fallback
        print("⚠️ Tentando processamento sequencial como fallback...")
        for i, key in enumerate(keys):
            try:
                all_addresses[i] = generate_bitcoin_address(key)
            except:
                pass

    elapsed = time.time() - start_time
    rate = n_keys / elapsed if elapsed > 0 else 0

    print(f"✅ Geração paralela concluída: {n_keys} endereços em {elapsed:.2f}s")
    print(f"📊 Taxa: {rate:.0f} endereços/s ({rate/1e6:.2f} Mend/s)")

    return all_addresses

def test_performance(n_keys=10000):
    """Executa um teste de desempenho da geração paralela de endereços."""
    print("=" * 60)
    print("🚀 TESTE DE DESEMPENHO - GERAÇÃO PARALELA DE ENDEREÇOS")
    print("=" * 60)

    # Gerar chaves aleatórias para teste
    print(f"🔄 Gerando {n_keys} chaves aleatórias...")
    keys = [int.from_bytes(np.random.bytes(32), 'big') % (2**256 - 2**32 - 977) + 1
            for _ in range(n_keys)]

    # Teste sequencial
    print("\n🔄 Teste sequencial:")
    start_time = time.time()
    addresses_seq = np.zeros((n_keys, 20), dtype=np.uint8)
    for i, key in enumerate(keys):
        addresses_seq[i] = generate_bitcoin_address(key)
    seq_elapsed = time.time() - start_time
    seq_rate = n_keys / seq_elapsed

    print(f"✅ Sequencial: {seq_elapsed:.2f}s ({seq_rate:.0f} end/s)")

    # Teste com diferentes números de workers
    for workers in [2, 4, 8, 16]:
        if workers > multiprocessing.cpu_count():
            continue  # Pular se não tivermos CPUs suficientes

        print(f"\n🔄 Teste paralelo com {workers} workers:")
        start_time = time.time()
        addresses_par = generate_addresses_parallel(keys, max_workers=workers)
        par_elapsed = time.time() - start_time
        par_rate = n_keys / par_elapsed

        # Verificar se os resultados são iguais
        matches = np.sum(np.all(addresses_seq == addresses_par, axis=1))
        accuracy = (matches / n_keys) * 100

        speedup = seq_elapsed / par_elapsed if par_elapsed > 0 else 0
        print(f"✅ Paralelo: {par_elapsed:.2f}s ({par_rate:.0f} end/s)")
        print(f"📊 Speedup: {speedup:.1f}x | Precisão: {accuracy:.2f}%")

    print("\n" + "=" * 60)

if __name__ == "__main__":
    # Executar teste de desempenho se for chamado diretamente
    test_performance(n_keys=20000)


In [ ]:
"""
Módulo auxiliar para processamento paralelo de endereços Bitcoin
Resolve o erro de pickling em funções aninhadas
"""

import numpy as np
from coincurve import PublicKey
import hashlib

# Funções de hashing têm que estar no escopo global para serem pickable
def sha256(data):
    """Calcula SHA-256"""
    return hashlib.sha256(data).digest()

def ripemd160(data):
    """Calcula RIPEMD-160 com suporte a diferentes implementações"""
    try:
        from Crypto.Hash import RIPEMD160
        h = RIPEMD160.new()
        h.update(data)
        return h.digest()
    except ImportError:
        # Fallback para hashlib se disponível
        try:
            h = hashlib.new('ripemd160')
            h.update(data)
            return h.digest()
        except:
            # Último recurso (não recomendado para produção)
            return hashlib.sha1(data).digest()

def bitcoin_hash160(public_key):
    """Implementação padrão Bitcoin: SHA-256 seguido de RIPEMD-160"""
    h = sha256(public_key)
    return ripemd160(h)

def process_keys_chunk(chunk_data):
    """
    Processa um conjunto de chaves em paralelo.

    Args:
        chunk_data: Tupla (keys, start_idx, end_idx)

    Returns:
        ndarray: Array NumPy contendo os hash160 dos endereços
    """
    keys, start_idx, end_idx = chunk_data
    chunk_size = end_idx - start_idx
    addresses = np.zeros((chunk_size, 20), dtype=np.uint8)

    # Pré-inicializar objetos para evitar recriação constante
    try:
        from Crypto.Hash import RIPEMD160
        ripemd = RIPEMD160.new
        has_pycrypto = True

        # Pré-inicializar função PublicKey para melhor performance
        from coincurve import PublicKey
        get_public_key = lambda priv_bytes: PublicKey.from_valid_secret(priv_bytes).format(compressed=False)[1:]

        # Pré-inicializar SHA256
        import hashlib
        sha256_func = hashlib.sha256
    except ImportError:
        has_pycrypto = False

    # Otimização: Processar em blocos para melhorar cache locality
    block_size = 128  # Tamanho do bloco

    for block_start in range(0, chunk_size, block_size):
        block_end = min(block_start + block_size, chunk_size)

        # Processar bloco
        for i in range(block_start, block_end):
            idx = i + start_idx
            if idx >= len(keys):
                break

            try:
                key = keys[idx]
                # Converter para bytes e calcular endereço
                key_hex = f"{key:064x}"
                pk_bytes = bytes.fromhex(key_hex)

                # Criar chave pública (sem copiar dados desnecessários)
                public_key = get_public_key(pk_bytes)

                # Hash SHA-256 otimizado
                h = sha256_func(public_key).digest()

                # RIPEMD-160 otimizado
                if has_pycrypto:
                    # Versão mais rápida com pycryptodome
                    r = ripemd()
                    r.update(h)
                    hash_bytes = r.digest()
                else:
                    hash_bytes = ripemd160(h)

                # Armazenar resultado sem cópias desnecessárias
                addresses[i] = np.frombuffer(hash_bytes, dtype=np.uint8)
            except Exception:
                # Manter zeros em caso de erro
                pass

    return addresses

def batch_generate_addresses(keys, max_workers=8, chunk_size=None):
    """
    Função wrapper para facilitar o uso do processamento paralelo

    Args:
        keys: Lista de chaves privadas
        max_workers: Número máximo de workers
        chunk_size: Tamanho de cada chunk (se None, calcula automaticamente)

    Returns:
        ndarray: Array NumPy (n_keys, 20) contendo os hash160 dos endereços
    """
    from concurrent.futures import ProcessPoolExecutor
    import multiprocessing
    import os

    # Definir variáveis de ambiente para melhorar performance das libs
    os.environ["OMP_NUM_THREADS"] = "1"  # Evitar que numpy crie threads em cada processo

    if max_workers is None or max_workers <= 0:
        max_workers = max(1, multiprocessing.cpu_count())

    batch_size = len(keys)
    addresses = np.zeros((batch_size, 20), dtype=np.uint8)

    # Calcular tamanho dos chunks - otimizado para reduzir overhead
    if chunk_size is None:
        # Para A100, mais workers com chunks menores funcionam melhor
        chunk_size = 2000  # Valor otimizado para A100

    # Preparar chunks para processamento
    chunks_data = []
    for start in range(0, batch_size, chunk_size):
        end = min(start + chunk_size, batch_size)
        chunks_data.append((keys, start, end))

    # Otimização: usar start_method='spawn' para evitar problemas de fork
    context = multiprocessing.get_context('spawn')

    # Processar em paralelo com um timeout maior e controle de falhas
    with ProcessPoolExecutor(max_workers=max_workers, mp_context=context) as executor:
        try:
            # Usar chunksize=1 para melhor balanceamento
            results = list(executor.map(process_keys_chunk, chunks_data, chunksize=1, timeout=180))
        except Exception as e:
            # Em caso de falha, processar sequencialmente
            print(f"⚠️ Falha no processamento paralelo: {e}")
            print(f"⚠️ Tentando método sequencial...")
            addresses = np.zeros((len(keys), 20), dtype=np.uint8)
            for i, key in enumerate(keys):
                try:
                    key_hex = f"{key:064x}"
                    pk_bytes = bytes.fromhex(key_hex)
                    public_key = PublicKey.from_valid_secret(pk_bytes).format(compressed=False)[1:]
                    hash_bytes = bitcoin_hash160(public_key)
                    addresses[i] = np.frombuffer(hash_bytes, dtype=np.uint8)
                except Exception:
                    pass  # Manter zeros em caso de erro
            return addresses

    # Unificar resultados
    for i, chunk_result in enumerate(results):
        start_idx = i * chunk_size
        end_idx = min(start_idx + chunk_size, batch_size)
        if start_idx < batch_size:  # Verificar limites
            actual_chunk_size = min(chunk_size, batch_size - start_idx)
            addresses[start_idx:end_idx] = chunk_result[:actual_chunk_size]

    return addresses


In [ ]:
!apt install batch_generate_addresses

In [ ]:
"""
Módulo para pré-computação de endereços Bitcoin em background
Implementa um modelo produtor-consumidor para reduzir o gargalo de CPU
"""
import threading
import queue
import time
import numpy as np
# from fixed_multiprocess import batch_generate_addresses
import multiprocessing

class AddressPrecomputer:
    """
    Classe para pré-computar endereços Bitcoin em threads de background.
    Usa um modelo produtor-consumidor para alimentar o processamento GPU.
    """
    def __init__(self, batch_size=32768, buffer_size=3, max_workers=None):
        """
        Inicializa o sistema de pré-computação de endereços.

        Args:
            batch_size: Tamanho de cada lote de endereços
            buffer_size: Número de lotes pré-computados a manter em buffer
            max_workers: Número máximo de workers para o processamento paralelo
        """
        self.batch_size = batch_size
        self.address_queue = queue.Queue(maxsize=buffer_size)
        self.stop_event = threading.Event()
        self.max_workers = max_workers or max(1, multiprocessing.cpu_count() - 1)
        self.producer_thread = None
        self.active = False

        # Estatísticas
        self.total_generated = 0
        self.total_consumed = 0
        self.start_time = 0

    def start(self, range_start, range_end):
        """
        Inicia o thread produtor para gerar endereços em background.

        Args:
            range_start: Início do intervalo de chaves
            range_end: Fim do intervalo de chaves
        """
        if self.active:
            return

        self.range_start = range_start
        self.range_end = range_end
        self.stop_event.clear()
        self.active = True
        self.start_time = time.time()

        # Iniciar thread produtor
        self.producer_thread = threading.Thread(
            target=self._producer_task,
            args=(range_start, range_end),
            daemon=True
        )
        self.producer_thread.start()

        print(f"✅ Iniciado pré-computador de endereços com buffer de {self.address_queue.maxsize} lotes")

    def stop(self):
        """Para o thread produtor e limpa o buffer."""
        self.stop_event.set()
        self.active = False

        # Esvaziar a fila
        while not self.address_queue.empty():
            try:
                self.address_queue.get_nowait()
                self.address_queue.task_done()
            except queue.Empty:
                break

    def _producer_task(self, range_start, range_end):
        """Tarefa de thread produtor que gera endereços continuamente."""
        # Calcular o tamanho do intervalo como bigint para evitar overflow
        sub_size = range_end - range_start + 1
        batch_idx = 0

        # Otimização: usar multiprocessing.set_start_method('spawn') para evitar problemas
        import multiprocessing
        try:
            multiprocessing.set_start_method('spawn', force=True)
        except RuntimeError:
            # Já foi configurado
            pass

        # Verificar e informar sobre o tamanho do intervalo
        range_too_large_for_uint32 = sub_size > 0xFFFFFFFF  # Maior que 2^32 - 1
        range_too_large_for_uint64 = sub_size > 0xFFFFFFFFFFFFFFFF  # Maior que 2^64 - 1

        if range_too_large_for_uint32:
            if range_too_large_for_uint64:
                print(f"⚠️ Range extremamente grande: {sub_size} (> uint64, usando método bytes)")
            else:
                print(f"⚠️ Range grande: {sub_size} (> uint32, usando uint64)")

        while not self.stop_event.is_set():
            try:
                # Verificar se a fila já está cheia
                if self.address_queue.qsize() >= self.address_queue.maxsize:
                    # Pausar brevemente se o buffer estiver cheio
                    time.sleep(0.5)
                    continue

                # Gerar chaves aleatórias - usando a estratégia apropriada para o tamanho do intervalo
                keys = []

                # Estratégia baseada no tamanho do intervalo
                if not range_too_large_for_uint32:
                    # CASO 1: Intervalo cabe em uint32 - usar método padrão randint
                    offsets = np.random.randint(0, sub_size, size=self.batch_size, dtype=np.uint32)
                    keys = [range_start + int(offset) for offset in offsets]

                elif not range_too_large_for_uint64:
                    # CASO 2: Intervalo maior que uint32 mas cabe em uint64
                    try:
                        # Primeiro, tentar usar uint64 diretamente
                        offsets = np.random.randint(0, sub_size, size=self.batch_size, dtype=np.uint64)
                        keys = [range_start + int(offset) for offset in offsets]
                    except OverflowError:
                        # Fallback para o método de bytes individuais
                        for _ in range(self.batch_size):
                            # Usar método bit a bit (até 64 bits)
                            max_bits = 64
                            random_bits = np.random.randint(0, 2, size=max_bits, dtype=np.uint8)
                            random_value = 0
                            for i, bit in enumerate(random_bits):
                                if bit:
                                    random_value |= (1 << i)

                            # Aplicar módulo para ficar no intervalo correto
                            random_value = random_value % sub_size
                            keys.append(range_start + random_value)

                else:
                    # CASO 3: Intervalo extremamente grande (> uint64)
                    for _ in range(self.batch_size):
                        # Usar método de bytes aleatórios (funciona para qualquer tamanho)
                        num_bytes = (sub_size.bit_length() + 7) // 8
                        num_bytes = max(num_bytes, 16)  # Pelo menos 16 bytes (128 bits)

                        # Gerar bytes aleatórios e converter para inteiro
                        random_bytes = np.random.bytes(num_bytes)
                        random_value = int.from_bytes(random_bytes, byteorder='big')

                        # Garantir que está dentro do intervalo
                        random_value = random_value % sub_size
                        keys.append(range_start + random_value)

                # Gerar endereços em paralelo
                start_time = time.time()
                addresses = batch_generate_addresses(
                    keys,
                    max_workers=self.max_workers,
                    chunk_size=2000  # Otimizado para A100
                )
                gen_time = time.time() - start_time

                # Adicionar ao buffer
                self.address_queue.put((keys, addresses, gen_time), block=True)

                self.total_generated += self.batch_size
                batch_idx += 1

                rate = self.batch_size / gen_time if gen_time > 0 else 0
                if batch_idx % 5 == 0:  # Log a cada 5 batches
                    print(f"🔄 Pré-computado lote {batch_idx}: {rate/1e6:.2f} Mend/s (buffer: {self.address_queue.qsize()}/{self.address_queue.maxsize})")

            except Exception as e:
                print(f"⚠️ Erro no produtor de endereços: {e}")
                import traceback
                traceback.print_exc()
                # Pausa para não sobrecarregar em caso de erros
                time.sleep(1.0)

    def get_next_batch(self, timeout=None):
        """
        Retorna o próximo lote de endereços pré-computados.

        Args:
            timeout: Tempo máximo de espera em segundos (None = esperar indefinidamente)

        Returns:
            tuple: (keys, addresses, generation_time) ou None se timeout
        """
        if not self.active:
            return None

        try:
            # Obter próximo lote (bloqueia até que um esteja disponível)
            next_batch = self.address_queue.get(block=True, timeout=timeout)
            self.address_queue.task_done()

            # Atualizar estatísticas
            self.total_consumed += self.batch_size

            # Mostrar estatísticas gerais
            elapsed = time.time() - self.start_time
            if elapsed > 0:
                avg_speed = self.total_consumed / elapsed / 1e6  # Mend/s
                print(f"📊 Taxa média: {avg_speed:.2f} Mend/s | Gerados: {self.total_generated:,} | Consumidos: {self.total_consumed:,}")

            return next_batch
        except queue.Empty:
            print("⚠️ Timeout ao aguardar por endereços pré-computados")
            return None

    def get_stats(self):
        """Retorna estatísticas do pré-computador."""
        elapsed = time.time() - self.start_time if self.start_time > 0 else 0
        return {
            "total_generated": self.total_generated,
            "total_consumed": self.total_consumed,
            "elapsed_time": elapsed,
            "average_speed": self.total_consumed / elapsed if elapsed > 0 else 0,
            "buffer_status": f"{self.address_queue.qsize()}/{self.address_queue.maxsize}"
        }


In [ ]:
!apt install multiprocessing

In [ ]:
"""
Script para criar manualmente os arquivos de módulo no Google Colab
"""
import os

def create_fixed_multiprocess():
    """Cria o arquivo fixed_multiprocess.py no diretório atual"""
    code = '''"""
Módulo auxiliar para processamento paralelo de endereços Bitcoin
Resolve o erro de pickling em funções aninhadas
"""

import numpy as np
from coincurve import PublicKey
import hashlib

# Funções de hashing têm que estar no escopo global para serem pickable
def sha256(data):
    """Calcula SHA-256"""
    return hashlib.sha256(data).digest()

def ripemd160(data):
    """Calcula RIPEMD-160 com suporte a diferentes implementações"""
    try:
        from Crypto.Hash import RIPEMD160
        h = RIPEMD160.new()
        h.update(data)
        return h.digest()
    except ImportError:
        # Fallback para hashlib se disponível
        try:
            h = hashlib.new('ripemd160')
            h.update(data)
            return h.digest()
        except:
            # Último recurso (não recomendado para produção)
            return hashlib.sha1(data).digest()

def bitcoin_hash160(public_key):
    """Implementação padrão Bitcoin: SHA-256 seguido de RIPEMD-160"""
    h = sha256(public_key)
    return ripemd160(h)

def process_keys_chunk(chunk_data):
    """
    Processa um conjunto de chaves em paralelo.

    Args:
        chunk_data: Tupla (keys, start_idx, end_idx)

    Returns:
        ndarray: Array NumPy contendo os hash160 dos endereços
    """
    keys, start_idx, end_idx = chunk_data
    chunk_size = end_idx - start_idx
    addresses = np.zeros((chunk_size, 20), dtype=np.uint8)

    # Pré-inicializar objetos para evitar recriação constante
    try:
        from Crypto.Hash import RIPEMD160
        ripemd = RIPEMD160.new
        has_pycrypto = True

        # Pré-inicializar função PublicKey para melhor performance
        from coincurve import PublicKey
        get_public_key = lambda priv_bytes: PublicKey.from_valid_secret(priv_bytes).format(compressed=False)[1:]

        # Pré-inicializar SHA256
        import hashlib
        sha256_func = hashlib.sha256
    except ImportError:
        has_pycrypto = False

    # Otimização: Processar em blocos para melhorar cache locality
    block_size = 128  # Tamanho do bloco

    for block_start in range(0, chunk_size, block_size):
        block_end = min(block_start + block_size, chunk_size)

        # Processar bloco
        for i in range(block_start, block_end):
            idx = i + start_idx
            if idx >= len(keys):
                break

            try:
                key = keys[idx]
                # Converter para bytes e calcular endereço
                key_hex = f"{key:064x}"
                pk_bytes = bytes.fromhex(key_hex)

                # Criar chave pública (sem copiar dados desnecessários)
                public_key = get_public_key(pk_bytes)

                # Hash SHA-256 otimizado
                h = sha256_func(public_key).digest()

                # RIPEMD-160 otimizado
                if has_pycrypto:
                    # Versão mais rápida com pycryptodome
                    r = ripemd()
                    r.update(h)
                    hash_bytes = r.digest()
                else:
                    hash_bytes = ripemd160(h)

                # Armazenar resultado sem cópias desnecessárias
                addresses[i] = np.frombuffer(hash_bytes, dtype=np.uint8)
            except Exception:
                # Manter zeros em caso de erro
                pass

    return addresses

def batch_generate_addresses(keys, max_workers=8, chunk_size=None):
    """
    Função wrapper para facilitar o uso do processamento paralelo

    Args:
        keys: Lista de chaves privadas
        max_workers: Número máximo de workers
        chunk_size: Tamanho de cada chunk (se None, calcula automaticamente)

    Returns:
        ndarray: Array NumPy (n_keys, 20) contendo os hash160 dos endereços
    """
    from concurrent.futures import ProcessPoolExecutor
    import multiprocessing
    import os

    # Definir variáveis de ambiente para melhorar performance das libs
    os.environ["OMP_NUM_THREADS"] = "1"  # Evitar que numpy crie threads em cada processo

    if max_workers is None or max_workers <= 0:
        max_workers = max(1, multiprocessing.cpu_count())

    batch_size = len(keys)
    addresses = np.zeros((batch_size, 20), dtype=np.uint8)

    # Calcular tamanho dos chunks - otimizado para reduzir overhead
    if chunk_size is None:
        # Para A100, mais workers com chunks menores funcionam melhor
        chunk_size = 2000  # Valor otimizado para A100

    # Preparar chunks para processamento
    chunks_data = []
    for start in range(0, batch_size, chunk_size):
        end = min(start + chunk_size, batch_size)
        chunks_data.append((keys, start, end))

    # Otimização: usar start_method='spawn' para evitar problemas de fork
    context = multiprocessing.get_context('spawn')

    # Processar em paralelo com um timeout maior e controle de falhas
    with ProcessPoolExecutor(max_workers=max_workers, mp_context=context) as executor:
        try:
            # Usar chunksize=1 para melhor balanceamento
            results = list(executor.map(process_keys_chunk, chunks_data, chunksize=1, timeout=180))
        except Exception as e:
            # Em caso de falha, processar sequencialmente
            print(f"⚠️ Falha no processamento paralelo: {e}")
            print(f"⚠️ Tentando método sequencial...")
            addresses = np.zeros((len(keys), 20), dtype=np.uint8)
            for i, key in enumerate(keys):
                try:
                    key_hex = f"{key:064x}"
                    pk_bytes = bytes.fromhex(key_hex)
                    public_key = PublicKey.from_valid_secret(pk_bytes).format(compressed=False)[1:]
                    hash_bytes = bitcoin_hash160(public_key)
                    addresses[i] = np.frombuffer(hash_bytes, dtype=np.uint8)
                except Exception:
                    pass  # Manter zeros em caso de erro
            return addresses

    # Unificar resultados
    for i, chunk_result in enumerate(results):
        start_idx = i * chunk_size
        end_idx = min(start_idx + chunk_size, batch_size)
        if start_idx < batch_size:  # Verificar limites
            actual_chunk_size = min(chunk_size, batch_size - start_idx)
            addresses[start_idx:end_idx] = chunk_result[:actual_chunk_size]

    return addresses
'''

    with open("fixed_multiprocess.py", "w") as f:
        f.write(code)

    print("✅ Arquivo fixed_multiprocess.py criado com sucesso no diretório atual.")

def create_address_precomputing():
    """Cria o arquivo address_precomputing.py no diretório atual"""
    code = '''"""
Módulo para pré-computação de endereços Bitcoin em background
Implementa um modelo produtor-consumidor para reduzir o gargalo de CPU
"""
import threading
import queue
import time
import numpy as np
from fixed_multiprocess import batch_generate_addresses
import multiprocessing

class AddressPrecomputer:
    """
    Classe para pré-computar endereços Bitcoin em threads de background.
    Usa um modelo produtor-consumidor para alimentar o processamento GPU.
    """
    def __init__(self, batch_size=32768, buffer_size=3, max_workers=None):
        """
        Inicializa o sistema de pré-computação de endereços.

        Args:
            batch_size: Tamanho de cada lote de endereços
            buffer_size: Número de lotes pré-computados a manter em buffer
            max_workers: Número máximo de workers para o processamento paralelo
        """
        self.batch_size = batch_size
        self.address_queue = queue.Queue(maxsize=buffer_size)
        self.stop_event = threading.Event()
        self.max_workers = max_workers or max(1, multiprocessing.cpu_count() - 1)
        self.producer_thread = None
        self.active = False

        # Estatísticas
        self.total_generated = 0
        self.total_consumed = 0
        self.start_time = 0

    def start(self, range_start, range_end):
        """
        Inicia o thread produtor para gerar endereços em background.

        Args:
            range_start: Início do intervalo de chaves
            range_end: Fim do intervalo de chaves
        """
        if self.active:
            return

        self.range_start = range_start
        self.range_end = range_end
        self.stop_event.clear()
        self.active = True
        self.start_time = time.time()

        # Iniciar thread produtor
        self.producer_thread = threading.Thread(
            target=self._producer_task,
            args=(range_start, range_end),
            daemon=True
        )
        self.producer_thread.start()

        print(f"✅ Iniciado pré-computador de endereços com buffer de {self.address_queue.maxsize} lotes")

    def stop(self):
        """Para o thread produtor e limpa o buffer."""
        self.stop_event.set()
        self.active = False

        # Esvaziar a fila
        while not self.address_queue.empty():
            try:
                self.address_queue.get_nowait()
                self.address_queue.task_done()
            except queue.Empty:
                break

    def _producer_task(self, range_start, range_end):
        """Tarefa de thread produtor que gera endereços continuamente."""
        # Calcular o tamanho do intervalo como bigint para evitar overflow
        sub_size = range_end - range_start + 1
        batch_idx = 0

        # Otimização: usar multiprocessing.set_start_method('spawn') para evitar problemas
        import multiprocessing
        try:
            multiprocessing.set_start_method('spawn', force=True)
        except RuntimeError:
            # Já foi configurado
            pass

        # Verificar tamanho do range para determinar a estratégia
        range_too_large = sub_size > 0xFFFFFFFF  # Maior que max uint32
        if range_too_large:
            print(f"⚠️ Range muito grande para uint32: {sub_size} (Usando método alternativo)")

        while not self.stop_event.is_set():
            try:
                # Verificar se a fila já está cheia
                if self.address_queue.qsize() >= self.address_queue.maxsize:
                    # Pausar brevemente se o buffer estiver cheio
                    time.sleep(0.5)
                    continue

                # Gerar chaves aleatórias - abordagem segura independente do tamanho
                keys = []
                np.random.seed()  # Renovar seed para melhor aleatoriedade

                # Método 100% seguro usando geração de números diretamente
                for _ in range(self.batch_size):
                    # Gerar um número aleatório no range [0, sub_size)
                    if range_too_large:
                        # Para intervalos muito grandes, gerar bytes aleatórios e convertê-los para um inteiro
                        # Calcular quantos bytes precisamos para representar sub_size
                        num_bytes = (sub_size.bit_length() + 7) // 8

                        # Gerar um valor aleatório usando bytes aleatórios
                        while True:
                            # Gerar bytes suficientes para cobrir o range
                            random_bytes = np.random.bytes(num_bytes)
                            random_value = int.from_bytes(random_bytes, byteorder='little')

                            # Aplicar módulo para ficar no range correto
                            random_value = random_value % sub_size

                            if random_value < sub_size:
                                break
                    else:
                        # Para intervalos menores, usar randint diretamente
                        random_value = np.random.randint(0, sub_size, dtype=np.uint64)

                    # Calcular chave final
                    key = range_start + random_value
                    keys.append(key)

                # Gerar endereços em paralelo
                start_time = time.time()
                addresses = batch_generate_addresses(
                    keys,
                    max_workers=self.max_workers,
                    chunk_size=2000  # Otimizado para A100
                )
                gen_time = time.time() - start_time

                # Adicionar ao buffer
                self.address_queue.put((keys, addresses, gen_time), block=True)

                self.total_generated += self.batch_size
                batch_idx += 1

                rate = self.batch_size / gen_time if gen_time > 0 else 0
                if batch_idx % 5 == 0:  # Log a cada 5 batches
                    print(f"🔄 Pré-computado lote {batch_idx}: {rate/1e6:.2f} Mend/s (buffer: {self.address_queue.qsize()}/{self.address_queue.maxsize})")

            except Exception as e:
                print(f"⚠️ Erro no produtor de endereços: {e}")
                import traceback
                traceback.print_exc()
                # Pausa para não sobrecarregar em caso de erros
                time.sleep(1.0)

    def get_next_batch(self, timeout=None):
        """
        Retorna o próximo lote de endereços pré-computados.

        Args:
            timeout: Tempo máximo de espera em segundos (None = esperar indefinidamente)

        Returns:
            tuple: (keys, addresses, generation_time) ou None se timeout
        """
        if not self.active:
            return None

        try:
            # Obter próximo lote (bloqueia até que um esteja disponível)
            next_batch = self.address_queue.get(block=True, timeout=timeout)
            self.address_queue.task_done()

            # Atualizar estatísticas
            self.total_consumed += self.batch_size

            # Mostrar estatísticas gerais
            elapsed = time.time() - self.start_time
            if elapsed > 0:
                avg_speed = self.total_consumed / elapsed / 1e6  # Mend/s
                print(f"📊 Taxa média: {avg_speed:.2f} Mend/s | Gerados: {self.total_generated:,} | Consumidos: {self.total_consumed:,}")

            return next_batch
        except queue.Empty:
            print("⚠️ Timeout ao aguardar por endereços pré-computados")
            return None

    def get_stats(self):
        """Retorna estatísticas do pré-computador."""
        elapsed = time.time() - self.start_time if self.start_time > 0 else 0
        return {
            "total_generated": self.total_generated,
            "total_consumed": self.total_consumed,
            "elapsed_time": elapsed,
            "average_speed": self.total_consumed / elapsed if elapsed > 0 else 0,
            "buffer_status": f"{self.address_queue.qsize()}/{self.address_queue.maxsize}"
        }
'''

    with open("address_precomputing.py", "w") as f:
        f.write(code)

    print("✅ Arquivo address_precomputing.py criado com sucesso no diretório atual.")

if __name__ == "__main__":
    # Verificar se os arquivos já existem
    if os.path.exists("fixed_multiprocess.py"):
        print("⚠️ O arquivo fixed_multiprocess.py já existe.")
        overwrite = input("Sobrescrever? (s/n): ").strip().lower() == 's'
        if overwrite:
            create_fixed_multiprocess()
    else:
        create_fixed_multiprocess()

    if os.path.exists("address_precomputing.py"):
        print("⚠️ O arquivo address_precomputing.py já existe.")
        overwrite = input("Sobrescrever? (s/n): ").strip().lower() == 's'
        if overwrite:
            create_address_precomputing()
    else:
        create_address_precomputing()

    print("\n✅ Módulos criados com sucesso!")
    print("   Agora você pode importá-los em seus scripts:")
    print("   from fixed_multiprocess import batch_generate_addresses")
    print("   from address_precomputing import AddressPrecomputer")


In [ ]:
from fixed_multiprocess import batch_generate_addresses
from address_precomputing import AddressPrecomputer


In [ ]:
"""
BitcoinFlix Miner - Versão CUDA para máximo desempenho GPU (Alternativa)
Usa CuPy para GPU sem depender das funções Numba que estão causando problemas de compatibilidade
"""

import os
import time
import sys
import requests
import numpy as np
import base58
from coincurve import PublicKey
from eth_utils import keccak
import hashlib
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading

# Detecta ambiente Colab
IS_COLAB = 'google.colab' in sys.modules

# Importar CuPy com tratamento de erro
print("🔄 Inicializando bibliotecas CUDA...")
try:
    import cupy as cp
    # Configurar LD_LIBRARY_PATH se necessário
    if 'google.colab' in sys.modules:
        import os
        cuda_link_dir = '/tmp/cuda_links'
        if os.path.exists(cuda_link_dir):
            current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
            if cuda_link_dir not in current_ld_path:
                os.environ['LD_LIBRARY_PATH'] = f"{cuda_link_dir}:{current_ld_path}"
                print(f"✅ LD_LIBRARY_PATH atualizado: {os.environ['LD_LIBRARY_PATH']}")

    HAS_CUDA = cp.cuda.is_available()
    if HAS_CUDA:
        # Obter informações da GPU usando CuPy
        dev_id = cp.cuda.Device()
        try:
            dev_props = cp.cuda.runtime.getDeviceProperties(dev_id.id)
            print(f"✅ CUDA disponível via CuPy: {dev_props['name'].decode()}")
            print(f"   - Memória total: {dev_props['totalGlobalMem'] / (1024**3):.2f} GB")
            print(f"   - Compute capability: {dev_props['major']}.{dev_props['minor']}")
            print(f"   - Multiprocessadores: {dev_props['multiProcessorCount']}")
        except Exception as e:
            # Fallback para informações básicas se houver erro
            print(f"✅ CUDA disponível via CuPy (informações limitadas)")
            print(f"   - Erro ao obter propriedades detalhadas: {e}")
            mem = cp.cuda.runtime.memGetInfo()
            print(f"   - Memória livre/total: {mem[0]/1024**3:.2f}GB/{mem[1]/1024**3:.2f}GB")
    else:
        print("❌ CUDA não está disponível")
except ImportError as e:
    HAS_CUDA = False
    print(f"⚠️ CuPy não encontrado: {e}")
    print("⚠️ Execute install_cuda_deps.py para instalar as dependências necessárias")

    # Auto-instalação das dependências
    if IS_COLAB:
        print("\n⚠️ Para corrigir os problemas CUDA, execute primeiro:")
        print("!python install_cuda_deps.py")
        print("E então reinicie o runtime (Runtime > Restart runtime) antes de executar este script novamente.")
        sys.exit(1)

# Importações para suporte RIPEMD160
try:
    from Crypto.Hash import RIPEMD160
    HAS_PYCRYPTO = True
except ImportError:
    HAS_PYCRYPTO = False
    print("⚠️ Crypto.Hash.RIPEMD160 não disponível, instalando pycryptodome...")
    try:
        import subprocess
        subprocess.run([sys.executable, "-m", "pip", "install", "pycryptodome"], check=True)
        from Crypto.Hash import RIPEMD160
        HAS_PYCRYPTO = True
        print("✅ pycryptodome instalado com sucesso")
    except:
        print("❌ Não foi possível instalar pycryptodome")
        HAS_PYCRYPTO = False

# Adicionar endereço da Golden Key
ADDITIONAL_ADDRESS = "1MVDYgVaSN6iKKEsbzRUAYFrYJadLYZvvZ"

# Configurações da API
POOL_TOKEN = "076a6a4636e1b70eb5105e609a2a9b59bcff5858f305daec5ee7b18095c6a48f"
API_URL = "https://bitcoinflix.replit.app/api/big_block"

# Configurações de processamento otimizadas com base nos resultados do teste de batch size
BATCH_SIZE = 32768 if HAS_CUDA else 8192     # Aumentado para melhorar ocupação da GPU
SUBBATCH_SIZE = 67108864 if HAS_CUDA else 2**20  # 64M chaves por sub-lote com GPU, 1M para CPU
GPU_MEMORY_FRACTION = 0.95    # Usar até 95% da memória GPU disponível
CPU_FALLBACK = False         # Flag para forçar uso da CPU mesmo com GPU disponível
MAX_PARALLEL_WORKERS = 32    # Usar mais workers para A100
PIPELINE_BUFFER_SIZE = 5     # Aumentar buffer de pipeline para manter GPU ocupada

# Parâmetros de otimização de GPU
GPU_BATCH_SIZE = 256        # Reduzido ainda mais para aumentar o número de kernels concorrentes
FORCE_GPU_SYNC = True        # Forçar sincronização ocasional para manter GPU ativa
GPU_WARMUP_ITERS = 10        # Número de iterações de aquecimento inicial
CUDA_STREAMS = 16            # Aumentado significativamente para maximum concurrency
GPU_STRESS_ENABLE = True     # Ativar operações de stress para aumentar clock da GPU
INTENSIVE_MATH_OPS = True    # Ativar operações matemáticas intensivas
CONTINUOUS_STRESS = True     # Ativar thread de stress contínuo
KERNEL_LOOPS = 50            # Aumentado drasticamente para mais operações por kernel
MAXIMUM_OCCUPANCY = True     # Nova flag para maximizar ocupação de SMs
CUDA_GRAPH_ENABLE = True     # Habilitar CUDA Graphs para kernels repetitivos
DISABLE_CACHING = True       # Desabilitar caching para forçar recomputação

# Agora podemos importar o módulo com segurança
from fixed_multiprocess import batch_generate_addresses

# Funções de utilidade
def sha256(data):
    """Calcula SHA-256"""
    return hashlib.sha256(data).digest()

def ripemd160(data):
    """Calcula RIPEMD-160 com suporte a diferentes implementações"""
    if HAS_PYCRYPTO:
        h = RIPEMD160.new()
        h.update(data)
        return h.digest()
    else:
        # Usar Keccak como fallback se RIPEMD160 não estiver disponível
        print("⚠️ Usando Keccak como fallback para RIPEMD160 (menos compatível)")
        return custom_keccak(data)[-20:]

def bitcoin_hash160(public_key):
    """Implementação correta do hash160 usado no Bitcoin (SHA256 + RIPEMD160)"""
    sha = sha256(public_key)
    ripe = ripemd160(sha)
    return ripe

def custom_keccak(data):
    """Calcula Keccak-256 com tratamento de erros."""
    try:
        if isinstance(data, str):
            data = bytes.fromhex(data.replace('0x', ''))
        return keccak(data)
    except Exception:
        return b'\x00' * 32

def decode_bitcoin_address(address):
    """Decodifica endereço Bitcoin para array numpy."""
    try:
        if not address or not isinstance(address, str):
            return None

        decoded = base58.b58decode(address)
        if len(decoded) != 25:
            return None

        hash_bytes = decoded[1:-4]
        hash_array = np.frombuffer(hash_bytes, dtype=np.uint8)
        return hash_array
    except Exception as e:
        print(f"⚠️ Erro ao decodificar {address}: {e}")
        return None

# Função para validar se um endereço Bitcoin é válido
def is_valid_bitcoin_address(address):
    """Verifica se um endereço Bitcoin é válido"""
    if not address or not isinstance(address, str):
        return False

    try:
        # Um endereço Bitcoin decodificado deve ter 25 bytes
        # (1 byte versão + 20 bytes hash + 4 bytes checksum)
        decoded = base58.b58decode(address)
        if len(decoded) != 25:
            return False

        # Verificar o checksum (os últimos 4 bytes)
        # Modificação: usar sha256(sha256(payload)) em vez de Keccak
        payload = decoded[:-4]
        checksum = sha256(sha256(payload))[:4]
        provided_checksum = decoded[-4:]

        return checksum == provided_checksum
    except Exception as e:
        print(f"⚠️ Erro na validação do endereço {address}: {e}")
        return False

# Função para buscar dados do bloco
def fetch_block_data():
    """Busca dados do bloco atual da API."""
    print("🔄 Buscando dados do bloco da API...")
    headers = {"pool-token": POOL_TOKEN}
    try:
        response = requests.get(API_URL, headers=headers)
        if response.status_code == 200:
            data = response.json()
            # Processar range
            range_data = data.get("range", {})
            start = range_data.get("start", "").replace("0x", "")
            end = range_data.get("end", "").replace("0x", "")
            print(f"✅ Range recebido: {start} até {end}")

            # Mostrar carteiras alvo
            addresses = data.get("checkwork_addresses", [])
            print(f"📋 Carteiras recebidas ({len(addresses)}):")
            for i, addr in enumerate(addresses):
                if addr:
                    print(f"  {i+1}. {addr}")

            return data
        else:
            print(f"❌ Erro: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"❌ Erro na requisição: {e}")
        return None

# Obter dados do bloco
BLOCK_DATA = fetch_block_data() or {
    "id": 483545,
    "position": 17895,
    "status": 0,
    "range": {
        "start": "0x9108ba3d21e522400",
        "end": "0x9108ba3d25e5223ff"
    },
    "checkwork_addresses": ["", ""],
    "message": "Retrieved existing block"
}

# Função para monitorar uso da GPU
def monitor_gpu():
    if not IS_COLAB or not HAS_CUDA:
        return

    try:
        import subprocess
        result = subprocess.run('nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total --format=csv,noheader',
                               shell=True, stdout=subprocess.PIPE).stdout.decode('utf-8').strip()
        gpu_util, mem_used, mem_total = result.split(',')
        print(f"📊 GPU: {gpu_util.strip()} | Memória: {mem_used.strip()}/{mem_total.strip()}")
    except:
        print("⚠️ Não foi possível monitorar GPU")

class CupyBitcoinMiner:
    """Minerador Bitcoin otimizado para CUDA usando apenas CuPy"""

    def __init__(self):
        """Inicializa o minerador"""
        self.current_block = BLOCK_DATA

        # Processa os targets
        self.targets_list = []
        self.target_addresses = []

        # Golden Key (endereço adicional)
        self.golden_key_address = ADDITIONAL_ADDRESS
        self.golden_key_hash = decode_bitcoin_address(ADDITIONAL_ADDRESS)
        if self.golden_key_hash is not None:
            print(f"🌟 Golden Key adicionada: {ADDITIONAL_ADDRESS}")

        # Decodificar endereços alvo
        for addr in self.current_block.get('checkwork_addresses', []):
            if isinstance(addr, str) and addr:
                # Validação adicional para confirmar que o endereço é válido
                if is_valid_bitcoin_address(addr):
                    self.target_addresses.append(addr)
                    hash_array = decode_bitcoin_address(addr)
                    if hash_array is not None:
                        self.targets_list.append(hash_array)
                else:
                    print(f"⚠️ Endereço inválido ignorado: {addr}")

        # Adicionar Golden Key aos targets se não estiver já incluída
        if self.golden_key_hash is not None:
            if not any(np.array_equal(self.golden_key_hash, target) for target in self.targets_list):
                self.targets_list.append(self.golden_key_hash)
                self.target_addresses.append(self.golden_key_address)
                print("✅ Golden Key adicionada aos targets")

        self.n_targets = len(self.targets_list)
        print(f"🎯 Total de targets válidos: {self.n_targets}")

        # Para uso com GPU, converter para arrays CuPy e otimizar memória
        if HAS_CUDA and self.n_targets > 0 and not CPU_FALLBACK:
            # Preparar array de targets para GPU
            self.targets_array_np = np.vstack(self.targets_list).astype(np.uint8)

            # Transferir para a GPU com CuPy
            self.targets_array = cp.asarray(self.targets_array_np)
            print("✅ Targets transferidos para GPU")

            # Pre-alocar buffers para reduzir fragmentação de memória
            try:
                # Obtém informação de memória disponível
                free_mem, total_mem = cp.cuda.runtime.memGetInfo()
                available_mem = int(free_mem * GPU_MEMORY_FRACTION)

                # Alocar buffers para endereços e resultados
                self.address_buffer = cp.zeros((BATCH_SIZE, 20), dtype=cp.uint8)
                self.result_buffer = cp.zeros(BATCH_SIZE, dtype=cp.int32)
                print(f"✅ Buffers pré-alocados: {BATCH_SIZE} endereços")

                # Ajustar batch size se necessário baseado na memória disponível
                meminfo_str = f"Memória GPU: {free_mem/(1024**3):.1f}GB livre de {total_mem/(1024**3):.1f}GB"
                print(f"📊 {meminfo_str}")
            except Exception as e:
                print(f"⚠️ Não foi possível pré-alocar buffers: {e}")

            # Aquecer a GPU para melhor desempenho
            self._warmup_gpu()
        else:
            self.targets_array_np = np.vstack(self.targets_list).astype(np.uint8) if self.targets_list else None
            self.targets_array = None

    def _warmup_gpu(self):
        """Aquece a GPU para melhor desempenho - versão extremamente agressiva"""
        if not HAS_CUDA or CPU_FALLBACK:
            return

        try:
            print("🔥 Aquecendo GPU e otimizando para desempenho máximo...")

            # Configurar GPU para desempenho máximo - ajustes extremos para memória
            try:
                # Desativar limites para computação máxima
                cp.cuda.runtime.deviceSetLimit(0, 8192)  # cudaLimitStackSize aumentado
                cp.cuda.runtime.deviceSetLimit(8, 128)   # cudaLimitMaxL2FetchGranularity aumentado

                # Configurações adicionais para maximizar throughput
                if hasattr(cp.cuda.runtime, 'deviceSetCacheConfig'):
                    cp.cuda.runtime.deviceSetCacheConfig(2)  # cudaFuncCachePreferL1

                # Aumentar tamanho de heap para operações dinâmicas
                if hasattr(cp.cuda.runtime, 'deviceSetLimit'):
                    cp.cuda.runtime.deviceSetLimit(1, 128*1024*1024)  # cudaLimitMallocHeapSize
            except:
                pass

            # Pré-alocar streams CUDA para operações paralelas
            self.cuda_streams = []
            self.cuda_events = []
            for i in range(CUDA_STREAMS):
                try:
                    # Criar streams não bloqueantes com alta prioridade
                    self.cuda_streams.append(cp.cuda.Stream(non_blocking=True, priority=0))

                    # Criar eventos para sincronização fina
                    self.cuda_events.append(cp.cuda.Event())
                except:
                    pass

            # WARMUP ULTRA AGRESSIVO: Executar kernels massivos em todas as streams
            print("   Executando warmup ultra agressivo para maximizar clocks...")

            # Criar gráficos CUDA para operações repetitivas
            if CUDA_GRAPH_ENABLE:
                try:
                    # Tentar configurar CUDA Graphs
                    self.cuda_graphs = []
                    self.cuda_graph_execs = []

                    # Criar um graph por stream para operações repetitivas
                    for stream in self.cuda_streams[:4]:  # Limitar a 4 graphs
                        with stream:
                            # Capturar um graph para operações comuns
                            graph = cp.cuda.graph.CUDAGraph()
                            with graph:
                                a = cp.random.normal(0, 1, (5000, 5000), dtype=cp.float32)
                                b = cp.random.normal(0, 1, (5000, 5000), dtype=cp.float32)
                                c = cp.matmul(a, b)
                                d = cp.exp(cp.sin(c) + cp.cos(c))
                                e = cp.linalg.cholesky(cp.matmul(d[:1000,:1000], d[:1000,:1000].T) + cp.eye(1000) * 0.01)

                            # Armazenar o graph e seu executor
                            self.cuda_graphs.append(graph)
                            self.cuda_graph_execs.append(graph.compile())

                    print("   ✅ CUDA Graphs configurados para execução rápida")
                except Exception as e:
                    print(f"   ⚠️ CUDA Graphs não disponíveis: {e}")

            # Iniciar thread separado para stress constante com alta prioridade
            if CONTINUOUS_STRESS:
                self.stress_thread_active = True
                self.stress_thread = threading.Thread(target=self._continuous_stress_thread, daemon=True)
                self.stress_thread.start()

            # AQUECIMENTO DE ALTA INTENSIDADE: Realizar operações em todas as streams em sequência
            for i in range(GPU_WARMUP_ITERS):
                # Para cada stream, executar operações diferentes
                for sidx, stream in enumerate(self.cuda_streams):
                    with stream:
                        # Tamanho diferente para cada stream para exercitar diferentes SMs
                        size = 5000 + (sidx % 5) * 1000

                        # Operações diferentes para exercitar diferentes unidades
                        if sidx % 4 == 0:
                            # GEMM (operações matriciais) - exercita unidades tensores
                            a = cp.random.random((size, size), dtype=cp.float32)
                            b = cp.random.random((size, size), dtype=cp.float32)
                            c = cp.matmul(a, b)
                            del a, b, c

                        elif sidx % 4 == 1:
                            # FFT (exercita unidades especiais)
                            size = min(size, 4096)  # FFT é mais pesado
                            a = cp.random.random((size, size), dtype=cp.float32)
                            b = cp.fft.fft2(a)
                            c = cp.fft.ifft2(b)
                            del a, b, c

                        elif sidx % 4 == 2:
                            # Elementwise operations (exercita CUDA cores)
                            n = size * size * 2
                            a = cp.random.random(n, dtype=cp.float32)
                            for _ in range(20):  # Loop intenso
                                a = cp.sin(a) * cp.cos(a)
                                a = cp.sqrt(cp.square(a) + 1.0)
                                a = cp.exp(a * 0.01)
                            del a

                        else:
                            # Operações de álgebra linear (exercita unidades específicas)
                            size = min(size, 3000)  # Álgebra linear é pesada
                            a = cp.random.random((size, size), dtype=cp.float32)
                            u, s, v = cp.linalg.svd(a, full_matrices=False)  # SVD é extremamente intensivo
                            del a, u, s, v

                    # Registrar evento para detectar conclusão
                    self.cuda_events[sidx].record(stream)

                # Sincronizar para garantir que todas as operações foram completadas
                for event in self.cuda_events:
                    event.synchronize()

                print(f"   Aquecimento intenso {i+1}/{GPU_WARMUP_ITERS} concluído")

            # Executar CUDA Graphs compilados para verificar desempenho
            if CUDA_GRAPH_ENABLE and hasattr(self, 'cuda_graph_execs') and self.cuda_graph_execs:
                print("   Executando CUDA Graphs compilados para teste de performance...")
                for i, graph_exec in enumerate(self.cuda_graph_execs):
                    start_time = time.time()
                    for _ in range(5):  # Executar 5 vezes para medir
                        graph_exec.launch()
                    cp.cuda.runtime.deviceSynchronize()
                    elapsed = time.time() - start_time
                    print(f"   Graph {i+1}: {elapsed/5*1000:.2f}ms por execução")

            # Liberar memória
            cp.get_default_memory_pool().free_all_blocks()
            print("✅ GPU totalmente preparada para desempenho máximo")

        except Exception as e:
            print(f"⚠️ Erro ao aquecer GPU: {e}")
            import traceback
            traceback.print_exc()

# Atualizar o runtime para usar mais CPU cores para processamento
def optimize_runtime_settings():
    """Otimiza configurações do runtime para melhor desempenho"""
    # Desativar GC durante o processamento intensivo
    import gc
    gc.disable()

    # Aumentar prioridade do processo
    try:
        os.nice(-10)  # Tenta aumentar prioridade em sistemas Unix
    except:
        pass

    # Configurar variáveis de ambiente para otimização
    os.environ["OMP_NUM_THREADS"] = str(MAX_PARALLEL_WORKERS)
    os.environ["MKL_NUM_THREADS"] = str(MAX_PARALLEL_WORKERS)
    os.environ["NUMEXPR_NUM_THREADS"] = str(MAX_PARALLEL_WORKERS)
    os.environ["OPENBLAS_NUM_THREADS"] = str(MAX_PARALLEL_WORKERS)
    os.environ["VECLIB_MAXIMUM_THREADS"] = str(MAX_PARALLEL_WORKERS)

    # Configurações específicas para CUDA
    if HAS_CUDA:
        # Configurar para desmapeamento agressivo de memória para evitar fragmentação
        os.environ["CUPY_GPU_MEMORY_LIMIT"] = "90%"
        os.environ["CUPY_MALLOC_MANAGED"] = "1"  # Usar malloc gerenciado

        # Ajuste fino para melhorar a persistência do kernel
        try:
            cp.cuda.runtime.setDeviceFlags(8)  # cudaDeviceScheduleYield
        except Exception:
            pass

        # Configurações para melhorar throughput
        try:
            # Desativar autosync
            cp.cuda.runtime.setDeviceFlags(4)  # cudaDeviceScheduleBlockingSync
        except Exception:
            pass

        # Configurações adicionais mais agressivas para CUDA
        os.environ["CUDA_LAUNCH_BLOCKING"] = "0"
        os.environ["CUDA_DEVICE_MAX_CONNECTIONS"] = "128"  # Aumentado para máxima concorrência
        os.environ["CUDA_CACHE_DISABLE"] = "1" if DISABLE_CACHING else "0"  # Forçar computação sem cache
        os.environ["CUDA_FORCE_PTX_JIT"] = "1"  # Forçar compilação JIT para otimização específica da GPU

        # Configurar para usar FMA (Fused Multiply-Add) de precisão simples
        os.environ["CUPY_FAST_MATH"] = "1"

        # Forçar modo de performance em GPUs NVIDIA - configurações mais agressivas
        try:
            import subprocess
            # Modo persistente
            subprocess.run("nvidia-smi -pm 1", shell=True,
                           stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            # Máxima frequência de memória e GPU - valores específicos para A100
            subprocess.run("nvidia-smi -ac 1215,1410", shell=True,
                           stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            # Sem limites de potência
            subprocess.run("nvidia-smi -pl 400", shell=True,
                           stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            # Prioridade máxima para o processo
            subprocess.run("nvidia-smi -c 3", shell=True,  # Modo COMPUTE_EXCLUSIVE
                           stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            print("🔥 GPU configurada para MÁXIMO DESEMPENHO")
        except Exception as e:
            print(f"ℹ️ Não foi possível definir configurações avançadas da GPU: {e}")

    # Contornar limitações de fork/subprocesso no Linux
    if sys.platform.startswith('linux'):
        import multiprocessing
        multiprocessing.set_start_method('spawn', force=True)

# Função principal
if __name__ == "__main__":
    if not HAS_CUDA:
        print("\n⚠️ CUDA não disponível. O processamento será feito na CPU (lento).")
        print("   Para usar GPU, verifique se:")
        print("   1. Ambiente está configurado para GPU (Runtime > Change runtime type)")
        print("   2. CuPy está instalado corretamente")

        if IS_COLAB:
            print("\nComo estamos no Colab, verifique se escolheu GPU em Runtime > Change runtime type")

        proceed = input("\nContinuar mesmo assim? (s/n): ")
        if proceed.lower() != 's':
            sys.exit(0)

    # Otimizar configurações de runtime antes de iniciar
    optimize_runtime_settings()

    miner = CupyBitcoinMiner()
    miner.run()


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda



In [ ]:
# ================== USO CPU ==================

import os
import torch
import time
import numpy as np
from coincurve import PublicKey
from eth_utils import keccak
import base58
import requests
import sys

# ================== CONFIGURAÇÃO ==================
os.environ['CUDA_HOME'] = '/usr/local/cuda'

POOL_TOKEN = "076a6a4636e1b70eb5105e609a2a9b59bcff5858f305daec5ee7b18095c6a48f"  # Token correto do pool
API_URL = "https://bitcoinflix.replit.app/api/big_block"  # URL da API

# Função para buscar automaticamente os dados do bloco da API
def fetch_block_data():
    print("🔄 Buscando dados do bloco da API...")
    headers = {"pool-token": POOL_TOKEN}
    try:
        response = requests.get(API_URL, headers=headers)
        if response.status_code == 200:
            data = response.json()
            # Log para depuração do range
            range_data = data.get("range", {})
            start = range_data.get("start", "").replace("0x", "")
            end = range_data.get("end", "").replace("0x", "")
            print(f"✅ Range recebido da API: start={start}, end={end}")

            # Exibir dados dos endereços recebidos
            addresses = data.get("checkwork_addresses", [])
            print(f"📋 Carteiras recebidas da API ({len(addresses)}):")
            for i, addr in enumerate(addresses):
                if addr: # Só mostra se não for vazio
                    print(f"  {i+1}. {addr}")

            # Exibir ID do bloco e outras informações relevantes
            print(f"🆔 Bloco ID: {data.get('id')}")
            print(f"📊 Posição: {data.get('position')}")

            return data
        else:
            print(f"❌ Erro ao buscar dados do bloco: {response.status_code} - {response.text}")
            return None
    except requests.RequestException as e:
        print(f"❌ Erro ao fazer a requisição: {e}")
        return None

# Função para obter informações da GPU
def get_gpu_info():
    """Obtém informações detalhadas sobre as GPUs disponíveis."""
    info = {
        "disponível": torch.cuda.is_available(),
        "dispositivos": 0,
        "modelo": "N/A",
        "memória_total_gb": 0
    }

    if info["disponível"]:
        try:
            info["dispositivos"] = torch.cuda.device_count()
            info["modelo"] = torch.cuda.get_device_name(0)
            info["memória_total_gb"] = torch.cuda.get_device_properties(0).total_memory / (1024**3)

            # Tenta obter informações adicionais com nvidia-smi
            try:
                import subprocess
                result = subprocess.run(["nvidia-smi", "--query-gpu=utilization.gpu,temperature.gpu", "--format=csv,noheader"],
                                       capture_output=True, text=True)
                if result.returncode == 0:
                    util, temp = result.stdout.strip().split(",")
                    info["utilização"] = util.strip()
                    info["temperatura"] = temp.strip()
            except:
                pass
        except Exception as e:
            print(f"Erro ao obter detalhes da GPU: {e}")

    return info

# Obtém dados do bloco automaticamente ou usa fallback estático
BLOCK_DATA = fetch_block_data() or {
    "id": 483545,
    "position": 17895,
    "status": 0,
    "range": {
        "start": "0x9108ba3d21e522400",
        "end": "0x9108ba3d25e5223ff"
    },
    "checkwork_addresses": [
        "",
        ""
    ],
    "message": "Retrieved existing unchecked block at position 17895"
}

SUBBATCH_SIZE = 2**23  # 262144 chaves por sub-lote; para testes, considere diminuir, ex: 2**16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PROGRESS_UPDATE_INTERVAL = 100000  # Atualiza o progresso a cada X chaves processadas

# ================== FUNÇÕES DE HASH ==================
def custom_keccak(data):
    """Calcula Keccak-256 com tratamento de erros."""
    try:
        if isinstance(data, str):
            data = bytes.fromhex(data.replace('0x', ''))
        return keccak(data)
    except Exception:
        return b'\x00' * 32

# ================== FUNÇÃO DE DECODIFICAÇÃO BASE58 ==================
def decode_bitcoin_address(address):
    """
    Decodifica um endereço Bitcoin em Base58Check e extrai os 20 bytes do hash.
    Um endereço P2PKH possui:
      - 1 byte de versão (0x00)
      - 20 bytes de hash
      - 4 bytes de checksum
    Retorna os 20 bytes do hash.
    """
    try:
        decoded = base58.b58decode(address)
        if len(decoded) != 25:
            return None
        return decoded[1:-4]
    except Exception as e:
        print(f"Erro ao decodificar {address}: {e}")
        return None

# ================== CLASSE DE MINERAÇÃO ==================
class BitcoinFlixMiner:
    def __init__(self):
        # Usando os dados dinâmicos obtidos da API
        self.current_block = BLOCK_DATA
        self.targets = {
            decode_bitcoin_address(addr)
            for addr in self.current_block.get('checkwork_addresses', [])
            if isinstance(addr, str) and addr
        }
        self.targets = {t for t in self.targets if t is not None}

        # Obter informações da GPU
        self.gpu_info = get_gpu_info()

        # Impressão de informações do dispositivo
        if self.gpu_info["disponível"]:
            print(f"🖥️ GPU: {self.gpu_info['modelo']}")
            print(f"📊 Memória: {self.gpu_info['memória_total_gb']:.2f} GB")
            if "utilização" in self.gpu_info:
                print(f"🌡️ Temperatura: {self.gpu_info['temperatura']}")
        else:
            print("⚠️ GPU não disponível, usando CPU")

    def process_range(self, start, end):
        """
        Processa um intervalo de chaves com informações detalhadas de progresso.
        """
        total_keys = end - start + 1
        valid_keys = []

        if total_keys <= 0:
            print("❌ Intervalo inválido (início >= fim)")
            return []

        num_subbatches = (total_keys + SUBBATCH_SIZE - 1) // SUBBATCH_SIZE
        print(f"🚀 Iniciando processamento: {num_subbatches} sub-lotes no intervalo")
        print(f"📊 Tamanho total do range: {total_keys:,} chaves")

        total_processed = 0
        global_start_time = time.time()

        for i in range(num_subbatches):
            sub_start = start + i * SUBBATCH_SIZE
            sub_end = min(sub_start + SUBBATCH_SIZE - 1, end)
            range_size = sub_end - sub_start + 1

            if range_size <= 0:
                continue

            batch_start_time = time.time()
            print(f"\n📌 [{i+1}/{num_subbatches}] Processando sub-lote {i+1}...")
            print(f"🔢 Range: {sub_start:x} até {sub_end:x} ({range_size:,} chaves)")

            # Gere offsets pequenos com np.uint32
            try:
                offsets = np.random.randint(0, range_size, size=SUBBATCH_SIZE, dtype=np.uint32)
                keys_cpu = [sub_start + int(off) for off in offsets]

                print("⚙️ Gerando endereços e verificando correspondências...")

                # Inicializa matriz para armazenar endereços
                addresses = np.zeros((SUBBATCH_SIZE, 20), dtype=np.uint8)

                # Processamento por lotes com atualização de progresso
                last_update_time = time.time()
                last_update_count = 0

                for j, key in enumerate(keys_cpu):
                    try:
                        pk_bytes = bytes.fromhex(f"{key:064x}")
                        public_key = PublicKey.from_valid_secret(pk_bytes).format(compressed=False)[1:]
                        addresses[j] = np.frombuffer(custom_keccak(public_key)[-20:], dtype=np.uint8)
                    except Exception:
                        addresses[j] = np.zeros(20, dtype=np.uint8)

                    # Atualização de progresso
                    if j % PROGRESS_UPDATE_INTERVAL == 0 and j > 0:
                        current_time = time.time()
                        elapsed = current_time - last_update_time
                        keys_since_update = j - last_update_count

                        if elapsed > 0:
                            speed = keys_since_update / elapsed
                            percent = (j / SUBBATCH_SIZE) * 100
                            eta = (SUBBATCH_SIZE - j) / speed if speed > 0 else 0

                            print(f"⏳ Progresso: {percent:.1f}% | {j:,}/{SUBBATCH_SIZE:,} chaves | "
                                  f"Velocidade: {speed/1e6:.2f} Mchaves/s | ETA: {eta:.1f}s")

                            last_update_time = current_time
                            last_update_count = j

                batch_time = time.time() - batch_start_time
                batch_speed = SUBBATCH_SIZE / batch_time if batch_time > 0 else 0
                print(f"✅ Geração concluída em {batch_time:.2f}s ({batch_speed/1e6:.2f} Mchaves/s)")

                # Verificação de correspondências
                print(f"🔍 Verificando correspondências com {len(self.targets)} carteiras...")
                match_start_time = time.time()

                if self.targets:
                    targets = np.array(list(self.targets), dtype=np.uint8)
                    matches = np.any(np.all(addresses[:, None] == targets, axis=2), axis=1)
                    matched_keys = [keys_cpu[j] for j, m in enumerate(matches) if m]
                    valid_keys.extend(matched_keys)

                match_time = time.time() - match_start_time
                print(f"✅ Verificação concluída em {match_time:.2f}s | Encontradas: {len(matched_keys)} chaves")

                # Atualiza estatísticas totais
                total_processed += SUBBATCH_SIZE
                total_elapsed = time.time() - global_start_time
                avg_speed = total_processed / total_elapsed if total_elapsed > 0 else 0

                print(f"📊 Estatísticas gerais:")
                print(f"   Processado: {total_processed:,}/{total_keys:,} chaves ({(total_processed/total_keys)*100:.1f}%)")
                print(f"   Velocidade média: {avg_speed/1e6:.2f} Mchaves/s")
                print(f"   Tempo decorrido: {total_elapsed:.2f}s")

                if len(valid_keys) >= 10:
                    print("🎯 Atingido limite de 10 chaves! Interrompendo processamento.")
                    break

            except Exception as e:
                print(f"❌ Erro no processamento do sub-lote {i+1}: {e}")

        # Estatísticas finais
        total_elapsed = time.time() - global_start_time
        final_speed = total_processed / total_elapsed if total_elapsed > 0 else 0
        print(f"\n📈 RESUMO FINAL:")
        print(f"   Processadas {total_processed:,} chaves em {total_elapsed:.2f}s")
        print(f"   Velocidade média: {final_speed/1e6:.2f} Mchaves/s")
        print(f"   Chaves válidas encontradas: {len(valid_keys)}")

        return valid_keys[:10]  # Retorna no máximo 10 chaves

    def submit_keys(self, keys):
        """
        Envia as chaves encontradas para a API com informações detalhadas.
        """
        if not keys:
            print("❌ Nenhuma chave para enviar.")
            return False

        print(f"\n📤 ENVIANDO {len(keys)} CHAVES PARA API")
        print("=" * 50)

        # Formata as chaves para o formato esperado pela API
        formatted_keys = []
        for i, key in enumerate(keys[:10]):
            # Converte para string hex e garante que tenha 64 caracteres
            hex_key = f"{key:064x}"
            # Adiciona o prefixo 0x
            formatted_key = f"0x{hex_key}"
            formatted_keys.append(formatted_key)
            print(f"  Chave #{i+1}: {formatted_key}")

        # Completa com zeros se não tiver 10 chaves
        remaining = 10 - len(formatted_keys)
        if remaining > 0:
            print(f"  + {remaining} chaves vazias para completar o lote de 10")
            for _ in range(remaining):
                formatted_keys.append("0x" + "0" * 64)

        # Prepara o payload para a API
        payload = {"privateKeys": formatted_keys}

        # Envia as chaves para a API
        headers = {
            "pool-token": POOL_TOKEN,
            "Content-Type": "application/json"
        }

        print("\n🔄 Enviando requisição para a API...")

        try:
            send_time = time.time()
            response = requests.post(API_URL, headers=headers, json=payload)
            elapsed = time.time() - send_time

            if response.status_code == 200:
                print(f"✅ SUCESSO! Chaves enviadas em {elapsed:.2f}s")
                try:
                    resp_data = response.json()
                    if resp_data:
                        print("📋 Resposta da API:")
                        for k, v in resp_data.items():
                            print(f"  {k}: {v}")
                except:
                    print("  Resposta sem dados JSON")
                return True
            else:
                print(f"❌ ERRO {response.status_code} ao enviar chaves ({elapsed:.2f}s)")
                print(f"📋 Resposta: {response.text}")
                return False

        except Exception as e:
            print(f"❌ ERRO na requisição: {e}")
            return False
        finally:
            print("=" * 50)

    def run(self):
        """Loop principal de mineração com informações detalhadas."""
        print("\n" + "="*60)
        print("🔥 BitcoinFlix Miner - Versão Otimizada com Progresso Detalhado 🔥")
        print("="*60)

        # Informações do dispositivo
        print(f"\n📱 INFORMAÇÕES DO DISPOSITIVO:")
        print(f"   Tipo: {'GPU' if torch.cuda.is_available() else 'CPU'}")
        if torch.cuda.is_available():
            print(f"   Modelo: {self.gpu_info['modelo']}")
            print(f"   Memória: {self.gpu_info['memória_total_gb']:.2f} GB")
            if "utilização" in self.gpu_info:
                print(f"   Utilização: {self.gpu_info['utilização']}")
                print(f"   Temperatura: {self.gpu_info['temperatura']}")

        # Informações do bloco
        print(f"\n📦 INFORMAÇÕES DO BLOCO:")
        print(f"   ID: {self.current_block['id']}")
        print(f"   Posição: {self.current_block.get('position', 'N/A')}")

        # Informações do range
        start_hex = self.current_block['range']['start']
        end_hex = self.current_block['range']['end']
        print(f"\n🔢 RANGE DE PROCESSAMENTO:")
        print(f"   Início: {start_hex}")
        print(f"   Fim: {end_hex}")

        try:
            range_start = int(start_hex, 16)
            range_end = int(end_hex, 16)
            range_size = range_end - range_start + 1
            print(f"   Tamanho: {range_size:,} chaves")
        except ValueError:
            print("❌ Erro ao converter intervalo para inteiro")
            return

        # Informações de configuração
        print(f"\n⚙️ CONFIGURAÇÕES:")
        print(f"   Tamanho do lote: {SUBBATCH_SIZE:,} chaves")
        print(f"   Alvos: {len(self.targets)} carteiras")

        print("\n" + "="*60)
        print("🚀 INICIANDO PROCESSAMENTO")
        print("="*60)

        start_time = time.time()
        valid_keys = self.process_range(range_start, range_end)

        elapsed = time.time() - start_time
        total_speed = SUBBATCH_SIZE * (range_size // SUBBATCH_SIZE) / elapsed if elapsed > 0 else 0

        print("\n" + "="*60)
        if valid_keys:
            print(f"🎯 RESULTADOS: {len(valid_keys)} CHAVES ENCONTRADAS")
            if self.submit_keys(valid_keys):
                print("✅ CHAVES ENVIADAS COM SUCESSO!")
            else:
                print("⚠️ ERRO NO ENVIO DAS CHAVES")
        else:
            print("😞 NENHUMA CHAVE VÁLIDA ENCONTRADA")

        # Estatísticas finais
        print("\n📊 ESTATÍSTICAS FINAIS:")
        print(f"   Tempo total: {elapsed:.2f} segundos")
        print(f"   Velocidade: {total_speed/1e6:.2f} Mchaves/segundo")
        print("="*60)

if __name__ == "__main__":
    miner = BitcoinFlixMiner()
    miner.run()